In [64]:
import pandas as pd
import warnings
import numpy as np

skaters_df=pd.read_csv('skaters_1996_2023.csv')
skaters_df.reset_index(drop=True, inplace=True)

goalies_df=pd.read_csv('goalies_1996_2023.csv')
goalies_df.reset_index(drop=True, inplace=True)
goalies_df=goalies_df.loc[goalies_df['SEASON']!=2021]
goalies_df

,RK,PLAYER,AGE,TM,GOALIE_STATS_GP,GOALIE_STATS_GS,GOALIE_STATS_W,GOALIE_STATS_L,GOALIE_STATS_T/O,GOALIE_STATS_GA,...,GOALIE_STATS_QS,GOALIE_STATS_QS%,GOALIE_STATS_RBS,GOALIE_STATS_GA%-,GOALIE_STATS_GSAA,SCORING_G,SCORING_A,SCORING_PTS,SCORING_PIM,SEASON
0,1,Jett Alexander,23,TOR,1,0,0,0,0,0,...,0,NaN,0,NaN,NaN,0,0,0,0,2023
1,2,Jake Allen,32,MTL,42,41,15,24,3,145,...,20,.488,10,113,-17.17,0,1,1,2,2023
2,3,Frederik Andersen,33,CAR,34,33,21,11,1,82,...,17,.515,6,101,-0.71,0,1,1,0,2023
3,4,Craig Anderson,41,BUF,26,24,11,11,2,76,...,12,.500,5,96,2.80,0,1,1,2,2023
4,5,Justus Annunen,22,COL,2,2,1,1,0,7,...,1,.500,1,NaN,NaN,0,0,0,0,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,74,Jimmy Waite,26,CHI,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1996
2451,75,Darcy Wakaluk,29,DAL,37,NaN,9,16,5,106,...,NaN,NaN,NaN,107,-6.96,0,0,0,6,1996
2452,76,Derek Wilkinson,21,TBL,4,NaN,0,3,0,15,...,NaN,NaN,NaN,NaN,NaN,0,0,0,2,1996
2453,77,Jordan Willis,20,DAL,1,NaN,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1996


In [108]:
dict_const_mapping={'FLA': 'Florida Panthers', 'TOR':'Toronto Maple Leafs', 'TBL': 'Tampa Bay Lightning', 'BOS': 'Boston Bruins', 'BUF':'Buffalo Sabres', 'DET': 'Detroit Red Wings', 'OTT':'Ottawa Senators','MTL': 'Montreal Canadiens', 'CAR': 'Carolina Hurricanes', 'NYR': 'New York Rangers', 'PIT': 'Pittsburgh Penguins', 'WSH': 'Washington Capitals', 'NYI': 'New York Islanders', 'CBJ': 'Columbus Blue Jackets', 'NJD': 'New Jersey Devils', 'PHI': 'Philadelphia Flyers' ,'COL': 'Colorado Avalanche', 'MIN': 'Minnesota Wild', 'STL': 'St. Louis Blues', 'DAL': 'Dallas Stars', 'NSH': 'Nashville Predators', 'WPG': 'Winnipeg Jets', 'CHI':'Chicago Blackhawks', 'ARI': 'Arizona Coyotes', 'CGY': 'Calgary Flames', 'EDM': 'Edmonton Oilers', 'LAK': 'Los Angeles Kings', 'VGK': 'Vegas Golden Knights', 'VAN': 'Vancouver Canucks', 'SJS': 'San Jose Sharks', 'ANA':'Anaheim Ducks'}

In [65]:
def positional_definition(value):
    if value=="C":
        return 2
    elif value=="RW":
        return 1
    elif value=="LW":
        return 3
    elif value=="F" or value=="W":
        return 3
    elif value=="D":
        return 8
    
def convert_atoi_to_seconds(value):
    minutes, seconds = map(int, value.split(':'))
    return minutes * 60 + seconds

In [10]:
key_features=['PLAYER', 'AGE', 'POS', 'SCORING_GP', 'SCORING_G', 'SCORING_A', 'SCORING_PTS', 'SCORING_+/-', 'SCORING_PIM','EV','SPECIAL_TEAMS_PP','SPECIAL_TEAMS_SH', 'ICE_TIME_ATOI', 'BLK' ,'HIT','SEASON', 'PS']
number_features=['AGE', 'POS', 'SCORING_GP', 'SCORING_G', 'SCORING_A', 'SCORING_PTS', 'SCORING_+/-', 'SCORING_PIM','EV','SPECIAL_TEAMS_PP','SPECIAL_TEAMS_SH', 'ICE_TIME_ATOI', 'BLK' ,'HIT','SEASON', 'PS']
normalized_features=['PLAYER', 'AGE', 'POS', 'SCORING_GP', 'G/GP', 'A/GP', 'PTS/GP', 'PM/GP', 'PIM/GP','EV/GP','PP/GP','SH/GP', 'ICE_TIME_ATOI', 'BLK/GP' ,'HIT/GP','SEASON', 'PS']
smaller_df=skaters_df[key_features]
smaller_df=smaller_df.dropna()
smaller_df=smaller_df.loc[smaller_df['AGE']!='Age']
smaller_df['POS']=smaller_df['POS'].apply(positional_definition)
smaller_df['ICE_TIME_ATOI']=smaller_df['ICE_TIME_ATOI'].apply(convert_atoi_to_seconds)
smaller_df[number_features]=smaller_df[number_features].astype(float)
smaller_df['G/GP']=smaller_df['SCORING_G']/smaller_df['SCORING_GP']
smaller_df['A/GP']=smaller_df['SCORING_A']/smaller_df['SCORING_GP']
smaller_df['PTS/GP']=smaller_df['SCORING_PTS']/smaller_df['SCORING_GP']
smaller_df['PM/GP']=smaller_df['SCORING_+/-']/smaller_df['SCORING_GP']
smaller_df['PIM/GP']=smaller_df['SCORING_PIM']/smaller_df['SCORING_GP']
smaller_df['EV/GP']=smaller_df['EV']/smaller_df['SCORING_GP']
smaller_df['PP/GP']=smaller_df['SPECIAL_TEAMS_PP']/smaller_df['SCORING_GP']
smaller_df['SH/GP']=smaller_df['SPECIAL_TEAMS_SH']/smaller_df['SCORING_GP']
smaller_df['BLK/GP']=smaller_df['BLK']/smaller_df['SCORING_GP']
smaller_df['HIT/GP']=smaller_df['HIT']/smaller_df['SCORING_GP']
normalized_df=smaller_df[normalized_features]
normalized_df

,PLAYER,AGE,POS,SCORING_GP,G/GP,A/GP,PTS/GP,PM/GP,PIM/GP,EV/GP,PP/GP,SH/GP,ICE_TIME_ATOI,BLK/GP,HIT/GP,SEASON,PS
0,Nicholas Abruzzese,23.0,2.0,2.0,0.000000,1.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.00,539.0,0.000000,0.500000,2023.0,0.2
1,Noel Acciari,31.0,2.0,77.0,0.181818,0.116883,0.298701,-0.051948,0.272727,0.181818,0.000000,0.00,859.0,1.051948,3.168831,2023.0,1.5
2,Calen Addison,22.0,8.0,62.0,0.048387,0.419355,0.467742,-0.274194,0.354839,0.048387,0.000000,0.00,967.0,0.612903,0.306452,2023.0,3.2
3,Egor Afanasyev,22.0,3.0,17.0,0.058824,0.000000,0.058824,-0.411765,0.117647,0.058824,0.000000,0.00,651.0,0.764706,1.352941,2023.0,-0.3
4,Andrew Agozzino,32.0,3.0,4.0,0.250000,0.500000,0.750000,-0.250000,0.000000,0.000000,0.000000,0.25,982.0,1.000000,3.000000,2023.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14352,Mike Zigomanis,27.0,2.0,33.0,0.060606,0.030303,0.090909,-0.212121,0.181818,0.060606,0.000000,0.00,744.0,0.363636,0.787879,2008.0,-0.4
14353,Ilya Zubov,20.0,2.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,878.0,0.000000,3.000000,2008.0,0.0
14354,Sergei Zubov*,37.0,8.0,46.0,0.086957,0.673913,0.760870,0.130435,0.260870,0.043478,0.043478,0.00,1542.0,0.891304,0.304348,2008.0,6.0
14355,Dainius Zubrus,29.0,2.0,82.0,0.158537,0.304878,0.463415,0.024390,0.463415,0.109756,0.048780,0.00,942.0,0.207317,0.939024,2008.0,3.7


In [11]:
key_features=['PLAYER', 'AGE', 'GOALIE_STATS_GP', 'GOALIE_STATS_W', 'GOALIE_STATS_L', 'GOALIE_STATS_T/O', 'GOALIE_STATS_GA','SV%','GOALIE_STATS_GAA','GOALIE_STATS_SO', 'SEASON', 'GPS']
number_features=['AGE', 'GOALIE_STATS_GP', 'GOALIE_STATS_W', 'GOALIE_STATS_L', 'GOALIE_STATS_T/O', 'GOALIE_STATS_GA','SV%','GOALIE_STATS_GAA','GOALIE_STATS_SO', 'SEASON', 'GPS']
normalized_features=['PLAYER', 'AGE', 'GOALIE_STATS_GP', 'GOALIE_STATS_W', 'GOALIE_STATS_L', 'GOALIE_STATS_T/O', 'GOALIE_STATS_GA','SV%','GOALIE_STATS_GAA','GOALIE_STATS_SO', 'SEASON', 'GPS']

smaller_goalie_df=goalies_df[key_features]
smaller_goalie_df=smaller_goalie_df.dropna()
smaller_goalie_df=smaller_goalie_df.loc[smaller_goalie_df['AGE']!='Age']
smaller_goalie_df[number_features]=smaller_goalie_df[number_features].astype(float)
smaller_goalie_df

,PLAYER,AGE,GOALIE_STATS_GP,GOALIE_STATS_W,GOALIE_STATS_L,GOALIE_STATS_T/O,GOALIE_STATS_GA,SV%,GOALIE_STATS_GAA,GOALIE_STATS_SO,SEASON,GPS
1,Jake Allen,32.0,42.0,15.0,24.0,3.0,145.0,0.891,3.55,1.0,2023.0,5.8
2,Frederik Andersen,33.0,34.0,21.0,11.0,1.0,82.0,0.903,2.48,1.0,2023.0,4.7
3,Craig Anderson,41.0,26.0,11.0,11.0,2.0,76.0,0.908,3.06,1.0,2023.0,4.9
4,Justus Annunen,22.0,2.0,1.0,1.0,0.0,7.0,0.854,3.58,0.0,2023.0,0.0
5,Yaroslav Askarov,20.0,1.0,0.0,1.0,0.0,4.0,0.886,4.15,0.0,2023.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
2450,Jimmy Waite,26.0,1.0,0.0,0.0,0.0,0.0,1.000,0.00,0.0,1996.0,0.1
2451,Darcy Wakaluk,29.0,37.0,9.0,16.0,5.0,106.0,0.891,3.39,1.0,1996.0,4.6
2452,Derek Wilkinson,21.0,4.0,0.0,3.0,0.0,15.0,0.857,4.50,0.0,1996.0,0.2
2453,Jordan Willis,20.0,1.0,0.0,1.0,0.0,1.0,0.929,3.20,0.0,1996.0,0.1


In [71]:
def find_nearest_neighbours(df, player, age):
    relevant_stats=['AGE', 'POS','SCORING_GP', 'G/GP','A/GP','PTS/GP','PM/GP','PIM/GP','EV/GP','PP/GP','SH/GP', 'ICE_TIME_ATOI', 'BLK/GP','HIT/GP']
    player_stats=df.loc[(df['AGE']==age) & (df['PLAYER']==player)].iloc[0]
    season=float(player_stats['SEASON'])
    player_stats=player_stats[relevant_stats]
    similar_ages=df.loc[(df['AGE']==age) & (df['PLAYER']!=player) & (df['SEASON']<=season)]
    similar_names=list(similar_ages['PLAYER'])
    point_shares=list(similar_ages['PS'])
    similar_ages=similar_ages[relevant_stats]
    differences=similar_ages.to_numpy()-player_stats.to_numpy()
    differences=pd.DataFrame(differences, columns=relevant_stats)
    if player_stats['POS']!=8:
        differences['similarity_score']=1000-(abs(differences['POS'])*300+abs(differences['ICE_TIME_ATOI'])*0.5+abs(differences['SCORING_GP'])*50+abs(differences['G/GP'])*400+abs(differences['A/GP'])*60+abs(differences['PTS/GP'])*250+abs(differences['PM/GP'])*1000+abs(differences['PIM/GP'])*10+abs(differences['EV/GP'])*40+abs(differences['PP/GP'])*1000+abs(differences['SH/GP'])*15+abs(differences['BLK/GP'])*10+abs(differences['HIT/GP'])*5)
    else:
        differences['similarity_score']=1000-(abs(differences['POS'])*300+abs(differences['ICE_TIME_ATOI'])*1+abs(differences['SCORING_GP'])*50+abs(differences['G/GP'])*100+abs(differences['A/GP'])*50+abs(differences['PTS/GP'])*100+abs(differences['PM/GP'])*2500+abs(differences['PIM/GP'])*50+abs(differences['EV/GP'])*40+abs(differences['PP/GP'])*30+abs(differences['SH/GP'])*15+abs(differences['BLK/GP'])*30+abs(differences['HIT/GP'])*30)
    differences['PLAYER']=similar_names
    differences['PS']=point_shares
    return differences.loc[differences['similarity_score']>0].sort_values(by='similarity_score', ascending=False), season

def find_nearest_neighbours_goalies(df, player, age):
    relevant_stats=['AGE', 'GOALIE_STATS_GP', 'GOALIE_STATS_W', 'GOALIE_STATS_L', 'GOALIE_STATS_T/O', 'GOALIE_STATS_GA','SV%','GOALIE_STATS_GAA','GOALIE_STATS_SO']
    player_stats=df.loc[(df['AGE']==age) & (df['PLAYER']==player)]
    season=float(player_stats['SEASON'])
    player_stats=player_stats[relevant_stats]
    similar_ages=df.loc[(df['AGE']==age) & (df['PLAYER']!=player) & (df['SEASON']<=season)]
    similar_names=list(similar_ages['PLAYER'])
    point_shares=list(similar_ages['GPS'])
    similar_ages=similar_ages[relevant_stats]
    # return similar_ages.to_numpy()
    differences=similar_ages.to_numpy()-player_stats.to_numpy()
    differences=pd.DataFrame(differences, columns=relevant_stats)
    differences['similarity_score']=1000-(abs(differences['GOALIE_STATS_GP'])*10+abs(differences['GOALIE_STATS_W'])*7+abs(differences['GOALIE_STATS_L'])*3+abs(differences['SV%'])*1000+abs(differences['GOALIE_STATS_GAA'])*100)
    differences['PLAYER']=similar_names
    differences['PS']=point_shares
    return differences.loc[differences['similarity_score']>0].sort_values(by='similarity_score', ascending=False), season

def determine_player_projections(df, player, age, number_of_years_from_now):
    stored, season=find_nearest_neighbours(df, player, age)
    list_names=list(stored['PLAYER'])
    stored_sim_scores=stored[['PLAYER','similarity_score']]
    projecting_df=df.loc[(df['PLAYER'].isin(list_names)) & (df['AGE']==age+number_of_years_from_now) & (df['SEASON']<=season)]
    projecting_df=projecting_df.merge(stored_sim_scores, how='left', on='PLAYER')
    similarity_score_sum=projecting_df['similarity_score'].sum()
    total=(projecting_df['similarity_score']*projecting_df['PS']).sum()
    projected_ps=total/similarity_score_sum
    return projected_ps

def determine_goalie_projections(df, player, age, number_of_years_from_now):
    stored, season=find_nearest_neighbours_goalies(df, player, age)
    list_names=list(stored['PLAYER'])
    stored_sim_scores=stored[['PLAYER','similarity_score']]
    projecting_df=df.loc[(df['PLAYER'].isin(list_names)) & (df['AGE']==age+number_of_years_from_now) & (df['SEASON']<=season)]
    projecting_df=projecting_df.merge(stored_sim_scores, how='left', on='PLAYER')
    similarity_score_sum=projecting_df['similarity_score'].sum()
    total=(projecting_df['similarity_score']*projecting_df['GPS']).sum()
    projected_ps=total/similarity_score_sum
    return projected_ps

def determine_player_max_age(player_name, df):
    player_rows=df.loc[df['PLAYER']==player_name]
    return float(max(player_rows['AGE']))

def construct_team_skater_strengths(dict_teams, df):
    dict_team_strengths={}
    for i in dict_teams:
        for j in dict_teams[i]:
            try:
                age=determine_player_max_age(j, df)
                if i not in dict_team_strengths:
                    dict_team_strengths[i]=max(0,determine_player_projections(df, j, age, 1))
                else:
                    dict_team_strengths[i]+=max(0,determine_player_projections(df, j, age, 1))
                print(determine_player_projections(normalized_df, j, age, 1))
            except:
                print(j)
    return dict_team_strengths

def construct_team_goalie_strengths(dict_teams_goalies, df):
    dict_team_strengths={}
    for i in dict_teams_goalies:
        for j in dict_teams_goalies[i]:
            try:
                age=determine_player_max_age(j, df)
                if i not in dict_team_strengths:
                    dict_team_strengths[i]=determine_goalie_projections(df, j, age, 1)
                else:
                    dict_team_strengths[i]+=determine_goalie_projections(df, j, age, 1)
                print(determine_goalie_projections(df, j, age, 1))
            except:
                print(j)
    return dict_team_strengths

def convert_to_elo_ratings(dict_vals):
    min_rating = min(dict_vals.values())
    max_rating = max(dict_vals.values())
    normalized_ratings = {
        team: ((rating - min_rating) / (max_rating - min_rating)) * (1600 - 1400) + 1400
        for team, rating in dict_vals.items()
    }
    current_average = sum(normalized_ratings.values()) / len(normalized_ratings)
    adjustment = 1500 - current_average
    adjusted_ratings = {team: rating + adjustment for team, rating in normalized_ratings.items()}
    return adjusted_ratings

In [24]:
team_colorado_2023=['Artturi Lehkonen', 'Nathan MacKinnon', 'Valeri Nichushkin', 'Tomáš Tatar', 'Ryan Johansen', 'Mikko Rantanen', 'Jonathan Drouin', 'Ross Colton', 'Miles Wood', 'Andrew Cogliano', 'Fredrik Olofsson', 'Logan O\'Connor', 'Devon Toews', 'Cale Makar', 'Bowen Byram', 'Samuel Girard', 'Jack Johnson', 'Josh Manson', 'Alexandar Georgiev', 'Pavel Francouz']
team_dallas_2023=['Jason Robertson', 'Roope Hintz', 'Joe Pavelski', 'Jamie Benn', 'Wyatt Johnston', 'Evgenii Dadonov', 'Mason Marchment', 'Matt Duchene', 'Tyler Seguin', 'Ty Dellandrea', 'Radek Faksa', 'Craig Smith', 'Miro Heiskanen', 'Ryan Suter', 'Thomas Harley', 'Jani Hakanpää', 'Nils Lundkvist', 'Esa Lindell', 'Jake Oettinger', 'Scott Wedgewood']
team_coyotes_2023=['Barrett Hayton', 'Nick Schmaltz', 'Clayton Keller', 'Matias Maccelli', 'Logan Cooley', 'Jan Jeník','Lawson Crouse', 'Nick Bjugstad', 'Zach Sanford', 'Jason Zucker', 'Alexander Kerfoot', 'Jack McBain', 'J.J. Moser', 'Sean Durzi', 'Mathew Dumba', 'Travis Dermott', 'Juuso Välimäki','Troy Stecher', 'Connor Ingram', 'Karel Vejmalka'] + ['Michael Carcone']
team_predators_2023=['Filip Forsberg', 'Ryan O\'Reilly', 'Juuso Pärssinen', 'Gustav Nyquist', 'Colton Sissons', 'Samuel Fagemo', 'Luke Evangelista', 'Thomas Novak', 'Kiefer Sherwood', 'Yakov Trenin', 'Michael McCarron', 'Cole Smith', 'Dante Fabbro', 'Roman Josi', 'Alexandre Carrier', 'Ryan McDonagh', 'Tyson Barrie', 'Jeremy Lauzon', 'Juuse Saros', 'Kevin Lankinen']
team_blues_2023=['Brandon Saad', 'Robert Thomas', 'Jordan Kyrou', 'Pavel Buchnevich', 'Brayden Schenn', 'Kasperi Kapanen', 'Jakub Vrána', 'Kevin Hayes', 'Sammy Blais', 'Jake Neighbours', 'Oskar Sundqvist', 'Alexey Toropchenko', 'Nick Leddy', 'Colton Parayko', 'Torey Krug', 'Justin Faulk', 'Tyler Tucker', 'Marco Scandella', 'Jordan Binnington', 'Joel Hofer']
team_wild_2023=['Kirill Kaprizov', 'Ryan Hartman', 'Mats Zuccarello', 'Pat Maroon', 'Joel Eriksson Ek', 'Marcus Johansson', 'Marcus Foligno', 'Marco Rossi', 'Frédérick Gaudreau', 'Matt Boldy', 'Connor Dewar', 'Brandon Duhaime', 'Jonas Brodin', 'Calen Addison', 'Jacob Middleton', 'Brock Faber', 'Jared Spurgeon', 'Jonathon Merrill', 'Filip Gustavsson', 'Marc-Andre Fleury']
team_jets_2023=['Kyle Connor', 'Mark Scheifele', 'Alex Iafallo', 'Nikolaj Ehlers', 'Vladislav Namestnikov', 'Cole Perfetti', 'Nino Niederreiter', 'Adam Lowry', 'Mason Appleton', 'David Gustafsson', 'Rasmus Kupari', 'Morgan Barron', 'Dylan DeMelo', 'Josh Morrissey', 'Brenden Dillon', 'Neal Pionk', 'Nate Schmidt', 'Dylan Samberg', 'Connor Hellebuyck', 'Laurent Brossoit']
team_blackhawks_2023=['Connor Bedard', 'Tyler Johnson', 'Nick Foligno', 'Andreas Athanasiou', 'Lukas Reichel', 'Taylor Raddysh', 'Corey Perry', 'Jason Dickinson', 'Philipp Kurashev', 'Boris Katchouk', 'Ryan Donato', 'Reese Johnson', 'Kevin Korchinski', 'Seth Jones', 'Alex Vlasic', 'Connor Murphy', 'Wyatt Kaiser', 'Jarred Tinordi', 'Petr Mrazek', 'Arvid Soderblom'] + ['Mackenzie Entwistle', 'Nikita Zaitsev']
team_knights_2023=['Ivan Barbashev', 'Jack Eichel', 'Jonathan Marchessault', 'Mark Stone', 'Chandler Stephenson', 'Paul Cotter', 'Brett Howden', 'William Karlsson', 'Michael Amadio', 'Keegan Kolesar', 'Nicolas Roy', 'William Carrier', 'Nicolas Hague', 'Alec Martinez', 'Shea Theodore', 'Brayden McNabb', 'Alex Pietrangelo', 'Brayden Pachal', 'Logan Thompson', 'Adin Hill']
team_canucks_2023=['Phillip Di Giuseppe', 'J.T. Miller', 'Brock Boeser', 'Andrei Kuzmenko', 'Elias Pettersson', 'Ilya Mikheyev', 'Dakota Joshua', 'Pius Suter', 'Conor Garland', 'Nils Höglander', 'Sam Lafferty', 'Anthony Beauvillier', 'Filip Hronek', 'Quinn Hughes', 'Ian Cole', 'Mark Friedman', 'Tyler Myers', 'Carson Soucy', 'Thatcher Demko', 'Casey Desmith']
team_kings_2023=['Anže Kopitar', 'Quinton Byfield', 'Adrian Kempe', 'Kevin Fiala', 'Pierre-Luc Dubois', 'Alex Laferriere', 'Arthur Kaliyev', 'Phillip Danault', 'Trevor Moore', 'Carl Grundström', 'Blake Lizotte', 'Trevor Lewis', 'Michael Anderson', 'Drew Doughty', 'Matt Roy', 'Vladislav Gavrikov', 'Jordan Spence', 'Andreas Englund', 'Phoenix Copley', 'Cam Talbot'] + ['Jaret Anderson-Dolan']
team_flames_2023=['Adam Ružička', 'Elias Lindholm', 'Matthew Coronato', 'Jonathan Huberdeau', 'Nazem Kadri', 'Dillon Dubé', 'Andrew Mangiapane', 'Mikael Backlund', 'Blake Coleman', 'Dryden Hunt', 'Yegor Sharangovich', 'Walker Duehr', 'Noah Hanifin', 'Chris Tanev', 'Nikita Zadorov', 'MacKenzie Weegar', 'Jordan Oesterle', 'Rasmus Andersson', 'Jacob Markstrom', 'Dan Vladar']
team_oilers_2023=['Leon Draisaitl', 'Evander Kane', 'Mattias Janmark', 'Zach Hyman', 'Ryan Nugent-Hopkins', 'Warren Foegele', 'Connor Brown', 'Ryan McLeod', 'Dylan Holloway', 'Adam Erne', 'Derek Ryan', 'Connor McDavid', 'Darnell Nurse', 'Cody Ceci', 'Mattias Ekholm', 'Evan Bouchard', 'Brett Kulak', 'Vincent Desharnais', 'Jack Campbell', 'Stuart Skinner']
team_kraken_2023=['Tye Kartye', 'Matthew Beniers', 'Jordan Eberle', 'Oliver Bjorkstrand', 'Alexander Wennberg', 'Jaden Schwartz', 'Eeli Tolvanen', 'Yanni Gourde', 'Jared McCann', 'Kailer Yamamoto', 'Pierre-Édouard Bellemare', 'André Burakovsky', 'Vince Dunn', 'Adam Larsson', 'William Borgen', 'Jamie Oleksiak', 'Justin Schultz', 'Brian Dumoulin', 'Joey Daccord', 'Philipp Grubauer'] + ['Brandon Tanev']
team_ducks_2023=['Adam Henrique', 'Trevor Zegras', 'Troy Terry', 'Ryan Strome', 'Mason McTavish', 'Frank Vatrano', 'Max Jones', 'Benoit-Olivier Groulx', 'Jakob Silfverberg', 'Brett Leason', 'Sam Carrick', 'Ross Johnston', 'Cam Fowler', 'Urho Vaakanainen', 'Radko Gudas', 'Ilya Lyubushkin', 'Pavel Mintyukov', 'Jackson LaCombe', 'John Gibson', 'Lukas Dostal'] + ['Robert Hagg']
team_sharks_2023=['Alexander Barabanov', 'Tomáš Hertl', 'Anthony Duclair', 'William Eklund', 'Jacob Peterson', 'Kevin Labanc', 'Fabian Zetterlund', 'Nico Sturm', 'Mike Hoffman', 'Filip Zadina', 'Thomas Bordeleau', 'Luke Kunin', 'Jan Rutta', 'Mario Ferraro', 'Matt Benning', 'Kyle Burroughs', 'Nikolai Knyzhov', 'Ty Emberson', 'Mackenzie Blackwood', 'Kaapo Kahkonen'] + ['Matt Benning']
team_bruins_2023=['Brad Marchand', 'Pavel Zacha', 'David Pastrňák', 'Morgan Geekie', 'Matthew Poitras', 'Jake DeBrusk', 'James van Riemsdyk', 'Charlie Coyle', 'Trent Frederic', 'Patrick Brown', 'John Beecher', 'Jakub Lauko', 'Matt Grzelcyk', 'Charlie McAvoy', 'Brandon Carlo', 'Hampus Lindholm', 'Derek Forbort', 'Kevin Shattenkirk', 'Jeremy Swayman', 'Linus Ullmark'] + ['Jakub Lauko', 'Jesper Boqvist']
team_wings_2023=['Alex DeBrincat', 'Dylan Larkin' , 'Lucas Raymond', 'Andrew Copp', 'J.T. Compher', 'Michael Rasmussen', 'David Perron', 'Joe Veleno', 'Daniel Sprong', 'Christian Fischer', 'Austin Czarnik', 'Jonatan Berggren', 'Jake Walman', 'Moritz Seider', 'Ben Chiarot', 'Justin Holl', 'Shayne Gostisbehere', 'Olli Määttä', 'Ville Husso', 'James Reimer']
team_canadiens_2023=['Cole Caufield', 'Nick Suzuki', 'Rafaël Harvey-Pinard', 'Juraj Slafkovsky', 'Alex Newhook', 'Tanner Pearson', 'Josh Anderson', 'Sean Monahan', 'Brendan Gallagher', 'Jesse Ylönen', 'Jake Evans', 'Michael Pezzetta', 'Mike Matheson', 'Johnathan Kovacevic', 'Jordan Harris', 'Kaiden Guhle', 'Justin Barron', 'Arber Xhekaj', 'Cayden Primeau', 'Jake Allen']
team_leafs_2023=['Calle Järnkrok', 'Auston Matthews', 'Mitch Marner', 'William Nylander', 'John Tavares', 'Tyler Bertuzzi', 'Matthew Knies', 'David Kämpf', 'Max Domi', 'Ryan Reaves', 'Pontus Holmberg', 'Noah Gregor', 'Morgan Rielly', 'T.J. Brodie', 'Timothy Liljegren', 'Jake McCabe', 'John Klingberg', 'Mark Giordano', 'Joseph Woll', 'Ilya Samsonov']
team_senators_2023=['Brady Tkachuk', 'Tim Stützle', 'Claude Giroux', 'Drake Batherson', 'Joshua Norris', 'Dominik Kubalík', 'Mathieu Joseph', 'Ridly Greig', 'Mark Kastelic', 'Rourke Chartier', 'Parker Kelly', 'Thomas Chabot', 'Jake Sanderson', 'Jakob Chychrun', 'Jacob Bernard-Docker', 'Travis Hamonic', 'Erik Brännström', 'Anton Forsberg', 'Joonas Korpisalo']
team_lightning_2023=['Alex Barré-Boulet', 'Brayden Point', 'Nikita Kucherov', 'Brandon Hagel', 'Nick Paul', 'Steven Stamkos', 'Tanner Jeannot', 'Anthony Cirelli', 'Waltteri Merela', 'Conor Sheary', 'Luke Glendening', 'Michael Eyssimont', 'Erik Černák', 'Victor Hedman', 'Mikhail Sergachev', 'Nicklaus Perbix', 'Darren Raddysh', 'Calvin de Haan', 'Jonas Johansson', ' Andrei Vasilevskiy'] + ['Austin Watson']
team_panthers_2023=['Evan Rodrigues','Eetu Luostarinen', 'Sam Reinhart', 'Matthew Tkachuk', 'Anton Lundell', 'Carter Verhaeghe', 'Kevin Stenlund', 'Aleksander Barkov', 'Nick Cousins', 'Steven Lorentz', 'Sam Bennett', 'Ryan Lomberg', 'Gustav Forsling', 'Oliver Ekman-Larsson', 'Dmitry Kulikov', 'Niko Mikkola', 'Josh Mahura', 'Aaron Ekblad', 'Brandon Montour', 'Sergei Bobrovsky','Anthony Stolarz'] 
team_sabres_2023=['Tage Thompson', 'Dylan Cozens', 'Jordan Greenway', 'Jeff Skinner', 'Casey Mittelstadt', 'Alex Tuch', 'Victor Olofsson', 'Peyton Krebs', 'John-Jason Peterka', 'Kyle Okposo', 'Tyson Jost', 'Zemgus Girgensons', 'Mattias Samuelsson', 'Rasmus Dahlin', 'Owen Power', 'Henri Jokiharju', 'Erik Johnson', 'Connor Clifton', 'Ukka-Pekka Luukkonen', 'Eric Comrie']
team_flyers_2023=['Sean Couturier', 'Cam Atkinson', 'Owen Tippett', 'Travis Konecny', 'Scott Laughton', 'Tyson Foerster', 'Bobby Brink', 'Noah Cates', 'Joel Farabee', 'Nicolas Deslauriers', 'Ryan Poehling', 'Garnet Hathaway', 'Cameron York', 'Travis Sanheim', 'Yegor Zamula', 'Sean Walker', 'Marc Staal', 'Rasmus Ristolainen', 'Carter Hart', 'Felix Sandstrom']
team_jackets_2023=['Jack Roslovic', 'Adam Fantilli', 'Alexandre Texier', 'Kirill Marchenko', 'Boone Jenner', 'Johnny Gaudreau', 'Kent Johnson', 'Cole Sillinger', 'Emil Bemström', 'Patrik Laine', 'Sean Kuraly', 'Justin Danforth', 'Zach Werenski', 'Damon Severson', 'David Jiricek', 'Ivan Provorov', 'Erik Gudbranson', 'Jake Bean', 'Elvis Merzlikins', 'Spencer Martin'] + ['Kirill Marchenko', 'Adam Boqvist']
team_rangers_2023=['Chris Kreider', 'Mika Zibanejad', 'Kaapo Kakko', 'Alexis Lafrenière', 'Filip Chytil', 'Artemi Panarin', 'Blake Wheeler', 'Vincent Trocheck', 'William Cuylle', 'Barclay Goodrow', 'Nick Bonino', 'Jimmy Vesey', 'Jacob Trouba', 'K\'Andre Miller', 'Adam Fox', 'Ryan Lindgren', 'Erik Gustafsson', 'Braden Schneider', 'Igor Shestyorkin', 'Jonathan Quick'] 
team_hurricanes_2023=['Sebastian Aho', 'Teuvo Teräväinen', 'Martin Nečas', 'Seth Jarvis', 'Jesperi Kotkaniemi', 'Stefan Noesen', 'Jordan Martinook', 'Jordan Staal', 'Jesper Fast', 'Brendan Lemieux', 'Jack Drury', 'Michael Bunting', 'Brent Burns', 'Jaccob Slavin', 'Brady Skjei', 'Jalen Chatfield', 'Anthony DeAngelo', 'Dmitry Orlov', 'Frederik Andersen', 'Antti Raanta']
team_devils_2023=['Jack Hughes', 'Timo Meier', 'Tyler Toffoli', 'Ondřej Palát', 'Nico Hischier', 'Jesper Bratt', 'Alexander Holtz', 'Erik Haula', 'Dawson Mercer', 'Curtis Lazar', 'Michael McLeod', 'Nathan Bastian', 'Dougie Hamilton', 'Jonas Siegenthaler', 'Kevin Bahl', 'John Marino', 'Luke Hughes', 'Brendan Smith', 'Akira Schmid', 'Vitek Vanecek']
team_islanders_2023=['Anders Lee', 'Bo Horvat', 'Mathew Barzal', 'Kyle Palmieri', 'Brock Nelson', 'Pierre Engvall', 'Simon Holmström', 'Jean-Gabriel Pageau', 'Oliver Wahlstrom', 'Cal Clutterbuck', 'Casey Cizikas', 'Matt Martin', 'Noah Dobson', 'Adam Pelech', 'Alexander Romanov', 'Ryan Pulock', 'Sebastian Aho', 'Samuel Bolduc', 'Ilya Sorokin', 'Semyon Varlamov']
team_penguins_2023=['Jake Guentzel', 'Sidney Crosby', 'Bryan Rust', 'Rickard Rakell', 'Evgeni Malkin', 'Reilly Smith', 'Radim Zohorna', 'Lars Eller', 'Drew O\'Connor', 'Lars Eller', 'Radim Zohorna', 'Jeff Carter', 'Noel Acciari', 'Matthew Nieto', 'Kris Letang', 'Ryan Graves', 'Marcus Pettersson', 'Erik Karlsson', 'John Ludvig', 'Ryan Shea', 'Alex Nedeljkovic', 'Tristan Jarry'] + ['Chad Ruhwedel', 'Pierre-Olivier Joseph']
team_capitals_2023=['Alex Ovechkin', 'Dylan Strome', 'Matthew Phillips', 'T.J. Oshie', 'Evgeny Kuznetsov', 'Connor McMichael', 'Sonny Milano', 'Nicklas Bäckström', 'Tom Wilson', 'Anthony Mantha', 'Nic Dowd', 'Beck Malenstyn', 'Martin Fehérváry', 'John Carlson', 'Trevor van Riemsdyk', 'Nick Jensen', 'Alexander Alexeyev', 'Rasmus Sandin', 'Darcy Kuemper', 'Hunter Shepard']

team_colorado_goalies_2023=['Alexandar Georgiev', 'Pavel Francouz']
team_dallas_goalies_2023=['Jake Oettinger', 'Scott Wedgewood']
team_coyotes_goalies_2023=['Connor Ingram', 'Karel Vejmelka']
team_predators_goalies_2023=['Juuse Saros', 'Kevin Lankinen']
team_blues_goalies_2023=['Jordan Binnington', 'Joel Hofer']
team_wild_goalies_2023=['Filip Gustavsson', 'Marc-André Fleury']
team_jets_goalies_2023=['Connor Hellebuyck', 'Laurent Brossoit']
team_blackhawks_goalies_2023=['Petr Mrázek', 'Arvid Söderblom']
team_knights_goalies_2023=['Logan Thompson', 'Adin Hill']
team_canucks_goalies_2023=['Thatcher Demko', 'Casey DeSmith']
team_kings_goalies_2023=['Pheonix Copley', 'Cam Talbot']
team_flames_goalies_2023=['Jacob Markström', 'Daniel Vladař']
team_oilers_goalies_2023=['Jack Campbell', 'Stuart Skinner']
team_kraken_goalies_2023=['Joey Daccord', 'Philipp Grubauer']
team_ducks_goalies_2023=['John Gibson', 'Lukáš Dostál']
team_sharks_goalies_2023=['Mackenzie Blackwood', 'Kaapo Kähkönen']
team_bruins_goalies_2023=['Jeremy Swayman', 'Linus Ullmark']
team_wings_goalies_2023=['Ville Husso', 'James Reimer']
team_canadiens_goalies_2023=['Cayden Primeau', 'Jake Allen']
team_leafs_goalies_2023=['Joseph Woll', 'Ilya Samsonov']
team_senators_goalies_2023=['Anton Forsberg', 'Joonas Korpisalo']
team_lightning_goalies_2023=['Jonas Johansson', 'Andrei Vasilevskiy']
team_panthers_goalies_2023=['Sergei Bobrovsky','Anthony Stolarz'] 
team_sabres_goalies_2023=['Ukko-Pekka Luukkonen', 'Eric Comrie']
team_flyers_goalies_2023=['Carter Hart', 'Felix Sandström']
team_jackets_goalies_2023=['Elvis Merzļikins', 'Spencer Martin']
team_rangers_goalies_2023=['Igor Shesterkin', 'Jonathan Quick'] 
team_hurricanes_goalies_2023=['Frederik Andersen', 'Antti Raanta']
team_devils_goalies_2023=['Akira Schmid', 'Vítek Vaněček']
team_islanders_goalies_2023=['Ilya Sorokin', 'Semyon Varlamov']
team_penguins_goalies_2023=['Alex Nedeljkovic', 'Tristan Jarry']
team_capitals_goalies_2023=['Darcy Kuemper', 'Hunter Shepard'] + ['Charlie Lindgren']

dict_teams_goalies_2023={'COL': team_colorado_goalies_2023, 'DAL': team_dallas_goalies_2023, 'ARI': team_coyotes_goalies_2023, 'NSH': team_predators_goalies_2023, 'STL': team_blues_goalies_2023, 'MIN': team_wild_goalies_2023, 'WPG': team_jets_goalies_2023, 'CHI': team_blackhawks_goalies_2023, 'VGK': team_knights_goalies_2023, 'VAN': team_canucks_goalies_2023, 'LAK': team_kings_goalies_2023, 'CGY': team_flames_goalies_2023, 'EDM': team_oilers_goalies_2023, 'SEA': team_kraken_goalies_2023, 'ANA': team_ducks_goalies_2023, 'SJS': team_sharks_goalies_2023, 'BOS': team_bruins_goalies_2023, 'DET': team_wings_goalies_2023, 'MTL': team_canadiens_goalies_2023, 'TOR': team_leafs_goalies_2023, 'OTT': team_senators_goalies_2023, 'TBL': team_lightning_goalies_2023, 'FLA': team_panthers_goalies_2023, 'BUF': team_sabres_goalies_2023, 'PHI': team_flyers_goalies_2023, 'CBJ': team_jackets_goalies_2023, 'NYR': team_rangers_goalies_2023, 'CAR': team_hurricanes_goalies_2023, 'NJD': team_devils_goalies_2023, 'NYI': team_islanders_goalies_2023, 'PIT': team_penguins_goalies_2023, 'WSH': team_capitals_goalies_2023}
dict_teams_2023={'COL': team_colorado_2023, 'DAL': team_dallas_2023, 'ARI': team_coyotes_2023, 'NSH': team_predators_2023, 'STL': team_blues_2023, 'MIN': team_wild_2023, 'WPG': team_jets_2023, 'CHI': team_blackhawks_2023, 'VGK': team_knights_2023, 'VAN': team_canucks_2023, 'LAK': team_kings_2023, 'CGY': team_flames_2023, 'EDM': team_oilers_2023, 'SEA': team_kraken_2023, 'ANA': team_ducks_2023, 'SJS': team_sharks_2023, 'BOS': team_bruins_2023, 'DET': team_wings_2023, 'MTL': team_canadiens_2023, 'TOR': team_leafs_2023, 'OTT': team_senators_2023, 'TBL': team_lightning_2023, 'FLA': team_panthers_2023, 'BUF': team_sabres_2023, 'PHI': team_flyers_2023, 'CBJ': team_jackets_2023, 'NYR': team_rangers_2023, 'CAR': team_hurricanes_2023, 'NJD': team_devils_2023, 'NYI': team_islanders_2023, 'PIT': team_penguins_2023, 'WSH': team_capitals_2023}

In [80]:
warnings.filterwarnings("ignore")

strengths_2023=construct_team_skater_strengths(dict_teams_2023, normalized_df)
team_goalie_strengths_2023=construct_team_goalie_strengths(dict_teams_goalies_2023, smaller_goalie_df)

dict_overall_team_strength_2023={}
for i in strengths_2023:
    dict_overall_team_strength_2023[i]=strengths_2023[i]+team_goalie_strengths_2023[i]

result_dict_2023 = {new_key: dict_overall_team_strength_2023[old_key] for old_key, new_key in dict_const_mapping.items()}
sorted_dict_team_strengths_2023 = dict(sorted(result_dict_2023.items(), key=lambda item: item[1], reverse=True))
elo_2023=convert_to_elo_ratings(sorted_dict_team_strengths_2023)

4.222631015887129
6.2319797168660855
5.457593903513477
4.484922448797589
2.561747704913828
6.286930938075679
1.5918617420678312
3.0690792068066957
2.309088533418448
2.229470226970277
0.7897052553747615
3.145965169973521
5.316670530233816
6.503448306131796
4.2803729927210075
4.617359529029255
5.028228175595368
Josh Manson
Alexandar Georgiev
Pavel Francouz
7.301596079788016
5.298969424226227
3.6620573556307128
3.478472161841564
4.872746504816292
1.5554379105533702
2.3048452132021704
3.5428623424982364
3.9101601487280644
4.0582042352431
2.676344460118952
1.0847993935548306
6.205523216195077
2.834630853275371
0.7128846942962658
3.6463689681350124
3.295875335559497
4.303741029222576
Jake Oettinger
Scott Wedgewood
4.604085694857332
4.48490600483167
5.490205513095324
2.8878237177919104
Logan Cooley
0.5815319139345505
4.12141922293258
2.9245976417490156
0.2770841412508828
3.5039659039164817
3.2308623735233506
3.5798822301630078
5.265307922346308
3.640535091663653
3.81283325330528
0.79288534504

In [96]:
team_colorado_2022=['Nathan MacKinnon', 'Mikko Rantanen', 'Valeri Nichushkin', 'Evan Rodrigues', 'J.T. Compher', 'Artturi Lehkonen', 'Andrew Cogliano', 'Logan O\'Connor', 'Alex Newhook', 'Denis Malgin', 'Matthew Nieto', 'Lars Eller', 'Devon Toews', 'Cale Makar', 'Samuel Girard', 'Josh Manson', 'Erik Johnson', 'Brad Hunt', 'Alexander Georgiev', 'Pavel Francouz'] + ['Samuel Girard']
team_dallas_2022=['Tyler Seguin', 'Jamie Benn', 'Jason Robertson', 'Joe Pavelski', 'Mason Marchment', 'Radek Faksa', 'Roope Hintz', 'Evgenii Dadonov', 'Luke Glendening', 'Wyatt Johnson', 'Ty Dellandrea', 'Miro Heiskanen','Esa Lindell', 'Ryan Suter', 'Colin Miller', 'Jani Hakanpää', 'Nils Lundkvist', 'Jake Oettinger', 'Scott Wedgewood'] + ['Mason Marchment']
team_coyotes_2022=['Clayton Keller', 'Nick Schmaltz', 'Lawson Crouse', 'Barrett Hayton', 'Travis Boyd', 'Christian Fischer', 'Jack McBain', 'Matias Maccelli', 'Liam O\'Brien', 'Brett Ritchie', 'Nick Bjugstad', 'Juuso Valimaki', 'Shayne Gostisbehere', 'Jakob Chychrun', 'Janis Moser', 'Conor Timmins', 'Troy Stetcher', 'Karel Vejmelka', 'Connor Ingram']
team_predators_2022=['Filip Forsberg', 'Cody Glass', 'Matt Duchene', 'Juuso Parssinen', 'Yakov Trenin', 'Nino Niederreiter', 'Ryan Johansen', 'Tommy Novak', 'Philip Tomasino', 'Zach Sanford', 'Tanner Jeannot', 'Mikael Granlund', 'Roman Josi', 'Dante Fabbro', 'Alexandre Carrier', 'Jordan Gross', 'Ryan McDonagh', 'Jeremy Lauzon', 'Juuse Saros', 'Kevin Lankinen']
team_blues_2022=['Robert Thomas', 'Jordan Kyrou', 'Brayden Schenn', 'Pavel Buchnevich', 'Ryan O\'Reilly', 'Ivan Barbashev', 'Vladimir Tarasenko', 'Josh Leivo', 'Brandon Saad', 'Noel Acciari', 'Tyler Pitlick', 'Brayden Schenn', 'Torey Krug', 'Colton Parayko', 'Calle Rosen', 'Nick Leddy', 'Justin Faulk', 'Niko Mikkola', 'Jordan Binnington', 'Thomas Greiss']
team_wild_2022=['Kirill Kaprizov', 'Mats Zuccarello', 'Matt Boldy', 'Joel Eriksson Ek', 'Sam Steel', 'Mason Shaw', 'Brandon Duhaime', 'Connor Dewar', 'Ryan Hartman', 'Marcus Foligno', 'Jordan Greenway', 'Ryan Reaves', 'Jonas Brodin', 'Calen Addison', 'Jon Merrill', 'Alex Goligoski', 'Jacob Middleton', 'Matt Dumba', 'Filip Gustavsson', 'Marc-Andre Fleury'] + ['Mathew Dumba', 'Jonathon Merrill']
team_jets_2022=['Mark Scheiffle', 'Pierre-Luc Dubois', 'Nikolaj Ehlers', 'Kyle Connor', 'Blake Wheeler', 'Jansen Harkins', 'Mason Appleton', 'Nino Niederreiter', 'Michael Eyssimont', 'Morgan Barron', 'Adam Lowry', 'Sam Gagner', 'Josh Morrissey', 'Dylan Samberg', 'Brenden Dillon', 'Nate Schmidt', 'Dylan DeMelo', 'Neal Pionk', 'Connor Hellebuyck', 'David Rittich']
team_blackhawks_2022=['Taylor Raddysh', 'Jujhar Khaira', 'Colin Blackwell', 'Boris Katchouk', 'Andreas Athanasiou', 'Sam Lafferty', 'Max Domi', 'Patrick Kane', 'Jason Dickinson', 'Jonathan Toews', 'Philipp Kurashev', 'Reese Johnson', 'Caleb Jones', 'Seth Jones', 'Connor Muphy', 'Jake McCabe', 'Jarred Tinordi', 'Jack Johnson', 'Petr Mrazek', 'Alex Stalock']
team_knights_2022=['Reilly Smith', 'Jack Eichel', 'Jonathan Marchessault', 'Mark Stone', 'Chandler Stephenson', 'Paul Cotter', 'Brett Howden', 'William Karlsson', 'Michael Amadio', 'Keegan Kolesar', 'Nicolas Roy', 'William Carrier', 'Nicolas Hague', 'Alec Martinez', 'Shea Theodore', 'Brayden McNabb', 'Alex Pietrangelo', 'Zach Whitecloud', 'Logan Thompson', 'Adin Hill']
team_canucks_2022=['Phillip Di Giuseppe', 'J.T. Miller', 'Brock Boeser', 'Andrei Kuzmenko', 'Elias Pettersson', 'Ilya Mikheyev', 'Dakota Joshua', 'Bo Horvat', 'Conor Garland', 'Nils Höglander', 'Nils Aman', 'Anthony Beauvillier', 'Filip Hronek', 'Quinn Hughes', 'Christian Wolanin', 'Travis Dermott', 'Tyler Myers', 'Kyle Burroughs', 'Thatcher Demko', 'Spencer Martin']
team_kings_2022=['Anže Kopitar', 'Quinton Byfield', 'Adrian Kempe', 'Kevin Fiala', 'Pierre-Luc Dubois', 'Alex Iafallo', 'Arthur Kaliyev', 'Phillip Danault', 'Trevor Moore', 'Carl Grundström', 'Blake Lizotte', 'Trevor Lewis', 'Michael Anderson', 'Drew Doughty', 'Matt Roy', 'Vladislav Gavrikov', 'Jordan Spence', 'Sean Durzi', 'Cal Peterson', 'Jonathan Quick'] + ['Jaret Anderson-Dolan']
team_flames_2022=['Adam Ružička', 'Elias Lindholm', 'Tyler Toffoli', 'Jonathan Huberdeau', 'Nazem Kadri', 'Dillon Dubé', 'Andrew Mangiapane', 'Mikael Backlund', 'Blake Coleman', 'Jakob Pelletier', 'Trevor Lewis', 'Walker Duehr', 'Noah Hanifin', 'Chris Tanev', 'Nikita Zadorov', 'MacKenzie Weegar', 'Troy Stetcher', 'Rasmus Andersson', 'Jacob Markstrom', 'Dan Vladar']
team_oilers_2022=['Leon Draisaitl', 'Evander Kane', 'Mattias Janmark', 'Zach Hyman', 'Ryan Nugent-Hopkins', 'Warren Foegele', 'Klim Kostin', 'Ryan McLeod', 'Dylan Holloway', 'Nick Bjugstad', 'Derek Ryan', 'Connor McDavid', 'Darnell Nurse', 'Cody Ceci', 'Mattias Ekholm', 'Evan Bouchard', 'Brett Kulak', 'Vincent Desharnais', 'Jack Campbell', 'Stuart Skinner']
team_kraken_2022=['Daniel Sprong', 'Matthew Beniers', 'Jordan Eberle', 'Oliver Bjorkstrand', 'Alexander Wennberg', 'Jaden Schwartz', 'Eeli Tolvanen', 'Yanni Gourde', 'Jared McCann', 'Morgan Geekie', 'Ryan Donato', 'André Burakovsky', 'Vince Dunn', 'Adam Larsson', 'William Borgen', 'Jamie Oleksiak', 'Justin Schultz', 'Carson Soucy', 'Joey Daccord', 'Philipp Grubauer'] + ['Brandon Tanev']
team_ducks_2022=['Adam Henrique', 'Trevor Zegras', 'Troy Terry', 'Ryan Strome', 'Mason McTavish', 'Frank Vatrano', 'Maxime Comtois', 'Benoit-Olivier Groulx', 'Jakob Silfverberg', 'Brett Leason', 'Sam Carrick', 'Isaac Lundestrom', 'Cam Fowler', 'Simon Benoit', 'Radko Gudas', 'Ilya Lyubushkin', 'Pavel Mintyukov', 'Jamie Drysdale', 'John Gibson', 'Lukas Dostal'] + ['Dmitri Kulikov']
team_sharks_2022=['Alexander Barabanov', 'Tomáš Hertl', 'Steven Lorentz', 'Timo Meier', 'Jacob Peterson', 'Kevin Labanc', 'Fabian Zetterlund', 'Nico Sturm', 'Logan Couture', 'Nick Bonino', 'Thomas Bordeleau', 'Luke Kunin', 'Erik Karlsson', 'Mario Ferraro', 'Matt Benning', 'Jaycob Megna', 'Nikolai Knyzhov', 'Marc-Eduard Vlasic', 'Mackenzie Blackwood', 'Kaapo Kahkonen'] + ['Matt Benning']
team_bruins_2022=['Brad Marchand', 'Pavel Zacha', 'David Pastrňák', 'Nick Foligno', 'Taylor Hall', 'Jake DeBrusk', 'Patrice Bergeron', 'Charlie Coyle', 'Trent Frederic', 'David Krejčí', 'Tomáš Nosek', 'Jakub Lauko', 'Matt Grzelcyk', 'Charlie McAvoy', 'Brandon Carlo', 'Hampus Lindholm', 'Derek Forbort', 'Mike Reilly', 'Jeremy Swayman', 'Linus Ullmark'] + ['Jakub Lauko', 'Jesper Boqvist']
team_wings_2022=['Pius Suter', 'Dylan Larkin' , 'Lucas Raymond', 'Andrew Copp', 'Tyler Bertuzzi', 'Michael Rasmussen', 'David Perron', 'Joe Veleno', 'Oskar Sundqvist', 'Filip Zadina', 'Austin Czarnik', 'Jonatan Berggren', 'Jake Walman', 'Moritz Seider', 'Filip Hronek', 'Robert Hagg', 'Jordan Oesterle', 'Olli Määttä', 'Ville Husso', 'Alex Nedeljkovic']
team_canadiens_2022=['Cole Caufield', 'Nick Suzuki', 'Rafaël Harvey-Pinard', 'Juraj Slafkovsky', 'Jonathan Drouin', 'Yevgeni Dadonov', 'Josh Anderson', 'Sean Monahan', 'Brendan Gallagher', 'Jesse Ylönen', 'Jake Evans', 'Michael Pezzetta', 'Mike Matheson', 'Johnathan Kovacevic', 'Jordan Harris', 'Kaiden Guhle', 'Justin Barron', 'Arber Xhekaj', 'Samuel Montembault', 'Jake Allen']
team_leafs_2022=['Calle Järnkrok', 'Auston Matthews', 'Mitch Marner', 'William Nylander', 'John Tavares', 'Pierre Engvall', 'Alexander Kerfoot', 'David Kämpf', 'Michael Bunting', 'Zach Aston-Reese', 'Pontus Holmberg', 'Wayne Simmonds', 'Morgan Rielly', 'T.J. Brodie', 'Timothy Liljegren', 'Jake McCabe', 'Rasmus Sandin', 'Mark Giordano', 'Matt Murray', 'Ilya Samsonov']
team_senators_2022=['Brady Tkachuk', 'Tim Stützle', 'Claude Giroux', 'Drake Batherson', 'Joshua Norris', 'Alex DeBrincat', 'Mathieu Joseph', 'Ridly Greig', 'Mark Kastelic', 'Shane Pinto', 'Austin Watson', 'Thomas Chabot', 'Jake Sanderson', 'Nikita Zaitsev', 'Jacob Bernard-Docker', 'Travis Hamonic', 'Erik Brännström', 'Artyom Zub', 'Cam Talbot', 'Anton Forsberg'] + ['Nikita Zaitsev', 'Derrick Brassard']
team_lightning_2022=['Alex Barré-Boulet', 'Brayden Point', 'Nikita Kucherov', 'Brandon Hagel', 'Nick Paul', 'Steven Stamkos', 'Corey Perry', 'Anthony Cirelli', 'Ross Colton', 'Alex Killorn', 'Pierre-Edouard Bellemare', 'Michael Eyssimont', 'Erik Černák', 'Victor Hedman', 'Mikhail Sergachev', 'Nicklaus Perbix', 'Darren Raddysh', 'Zach Bogosian', 'Brian Elliott', ' Andrei Vasilevskiy'] + ['Austin Watson']
team_panthers_2022=['Eric Staal','Eetu Luostarinen', 'Sam Reinhart', 'Matthew Tkachuk', 'Anton Lundell', 'Carter Verhaeghe', 'Ryan Lomberg', 'Aleksander Barkov', 'Nick Cousins', 'Anthony Duclair', 'Sam Bennett', 'Ryan Lomberg', 'Gustav Forsling', 'Radko Gudas', 'Marc Staal', 'Josh Mahura', 'Aaron Ekblad', 'Brandon Montour', 'Sergei Bobrovsky','Anthony Stolarz'] 
team_sabres_2022=['Tage Thompson', 'Dylan Cozens', 'Zemgus Girgensons', 'Jeff Skinner', 'Casey Mittelstadt', 'Alex Tuch', 'Victor Olofsson', 'Peyton Krebs', 'John-Jason Peterka', 'Kyle Okposo', 'Tyson Jost', 'Jack Quinn', 'Mattias Samuelsson', 'Rasmus Dahlin', 'Owen Power', 'Henri Jokiharju', 'Jacob Bryson', 'Kale Clague', 'Ukka-Pekka Luukkonen', 'Eric Comrie']
team_flyers_2022=['James van Riemsdyk', 'Cam Atkinson', 'Owen Tippett', 'Travis Konecny', 'Scott Laughton', 'Tyson Foerster', 'Morgan Frost', 'Noah Cates', 'Joel Farabee', 'Nicolas Deslauriers', 'Patrick Brown', 'Brendan Lemieux', 'Cameron York', 'Travis Sanheim', 'Anthony DeAngelo', 'Ivan Provorov', 'Justin Braun', 'Rasmus Ristolainen', 'Carter Hart', 'Felix Sandstrom']
team_jackets_2022=['Jack Roslovic', 'Liam Foudy', 'Yegor Chinakov', 'Kirill Marchenko', 'Boone Jenner', 'Johnny Gaudreau', 'Kent Johnson', 'Cole Sillinger', 'Emil Bemström', 'Patrik Laine', 'Sean Kuraly', 'Justin Danforth', 'Zach Werenski', 'Adam Boqvist', 'Nick Blankenburg', 'Vladislav Gavrikov', 'Erik Gudbranson', 'Jake Bean', 'Elvis Merzlikins', 'Joonas Korpisalo'] + ['Kirill Marchenko', 'Adam Boqvist']
team_rangers_2022=['Chris Kreider', 'Mika Zibanejad', 'Kaapo Kakko', 'Alexis Lafrenière', 'Filip Chytil', 'Artemi Panarin', 'Vitali Kravtsov', 'Vincent Trocheck', 'William Cuylle', 'Barclay Goodrow', 'Sammy Blais', 'Jimmy Vesey', 'Jacob Trouba', 'K\'Andre Miller', 'Adam Fox', 'Ryan Lindgren', 'Ben Harpur', 'Braden Schneider', 'Igor Shestyorkin', 'Jonathan Quick'] 
team_hurricanes_2022=['Sebastian Aho', 'Teuvo Teräväinen', 'Martin Nečas', 'Seth Jarvis', 'Jesperi Kotkaniemi', 'Stefan Noesen', 'Jordan Martinook', 'Jordan Staal', 'Jesper Fast', 'Andrei Svechnikov', 'Jack Drury', 'Paul Stasny', 'Brent Burns', 'Jaccob Slavin', 'Brady Skjei', 'Jalen Chatfield', 'Tony DeAngelo', 'Brett Pesce', 'Frederik Andersen', 'Antti Raanta']
team_devils_2022=['Jack Hughes', 'Tomáš Tatar', 'Yegor Sharangovich', 'Ondřej Palát', 'Nico Hischier', 'Jesper Bratt', 'Alexander Holtz', 'Erik Haula', 'Dawson Mercer', 'Curtis Lazar', 'Michael McLeod', 'Nathan Bastian', 'Dougie Hamilton', 'Jonas Siegenthaler', 'Kevin Bahl', 'John Marino', 'Ryan Graves', 'Brendan Smith', 'Mackenzie Blackwood', 'Vitek Vanecek']
team_islanders_2022=['Anders Lee', 'Anthony Beauvillier', 'Mathew Barzal', 'Kyle Palmieri', 'Brock Nelson', 'Hudson Fasching', 'Simon Holmström', 'Jean-Gabriel Pageau', 'Oliver Wahlstrom', 'Cal Clutterbuck', 'Casey Cizikas', 'Matt Martin', 'Noah Dobson', 'Adam Pelech', 'Alexander Romanov', 'Ryan Pulock', 'Sebastian Aho', 'Samuel Bolduc', 'Ilya Sorokin', 'Semyon Varlamov']
team_penguins_2022=['Jake Guentzel', 'Sidney Crosby', 'Bryan Rust', 'Rickard Rakell', 'Evgeni Malkin', 'Ryan Poehling', 'Teddy Blueger', 'Danton Heinen', 'Drew O\'Connor', 'Brock McGinn', 'Mikael Granlund', 'Jeff Carter', 'Chad Ruhwedel', 'Kasperi Kapanen', 'Kris Letang', 'Brian Dumoulin', 'Marcus Pettersson', 'Pierre-Olivier Joseph', 'Jan Rutta', 'Jeff Petry', 'Casey DeSmith', 'Tristan Jarry'] + ['Chad Ruhwedel', 'Pierre-Olivier Joseph']
team_capitals_2022=['Alex Ovechkin', 'Dylan Strome', '', 'T.J. Oshie', 'Evgeny Kuznetsov', 'Connor McMichael', 'Sonny Milano', 'Nicklas Bäckström', 'Tom Wilson', 'Anthony Mantha', 'Nic Dowd', 'Beck Malenstyn', 'Martin Fehérváry', 'John Carlson', 'Trevor van Riemsdyk', 'Nick Jensen', 'Alexander Alexeyev', 'Matt Irwin', 'Darcy Kuemper', 'Charlie Lindgren']

team_colorado_goalies_2022=['Alexandar Georgiev', 'Pavel Francouz']
team_dallas_goalies_2022=['Jake Oettinger', 'Scott Wedgewood']
team_coyotes_goalies_2022=['Connor Ingram', 'Karel Vejmelka']
team_predators_goalies_2022=['Juuse Saros', 'Kevin Lankinen']
team_blues_goalies_2022=['Jordan Binnington', 'Thomas Greiss']
team_wild_goalies_2022=['Filip Gustavsson', 'Marc-André Fleury']
team_jets_goalies_2022=['Connor Hellebuyck', 'David Rittich']
team_blackhawks_goalies_2022=['Petr Mrázek', 'Alex Stalock']
team_knights_goalies_2022=['Logan Thompson', 'Adin Hill']
team_canucks_goalies_2022=['Thatcher Demko', 'Spencer Martin']
team_kings_goalies_2022=['Jonathan Quick', 'Calvin Petersen']
team_flames_goalies_2022=['Jacob Markström', 'Daniel Vladař']
team_oilers_goalies_2022=['Jack Campbell', 'Stuart Skinner']
team_kraken_goalies_2022=['Joey Daccord', 'Philipp Grubauer']
team_ducks_goalies_2022=['John Gibson', 'Lukáš Dostál']
team_sharks_goalies_2022=['James Reimer', 'Kaapo Kähkönen']
team_bruins_goalies_2022=['Jeremy Swayman', 'Linus Ullmark']
team_wings_goalies_2022=['Ville Husso', 'Alex Nedeljkovic']
team_canadiens_goalies_2022=['Sam Montembeault', 'Jake Allen']
team_leafs_goalies_2022=['Joseph Woll', 'Ilya Samsonov']
team_senators_goalies_2022=['Anton Forsberg', 'Cam Talbot']
team_lightning_goalies_2022=['Brian Elliott', 'Andrei Vasilevskiy']
team_panthers_goalies_2022=['Sergei Bobrovsky','Spencer Knight'] 
team_sabres_goalies_2022=['Ukko-Pekka Luukkonen', 'Eric Comrie']
team_flyers_goalies_2022=['Carter Hart', 'Felix Sandström']
team_jackets_goalies_2022=['Elvis Merzļikins', 'Spencer Martin']
team_rangers_goalies_2022=['Igor Shesterkin', 'Jaroslav Halák'] 
team_hurricanes_goalies_2022=['Frederik Andersen', 'Antti Raanta']
team_devils_goalies_2022=['Mackenzie Blackwood', 'Vítek Vaněček']
team_islanders_goalies_2022=['Ilya Sorokin', 'Semyon Varlamov']
team_penguins_goalies_2022=['Casey DeSmith', 'Tristan Jarry']
team_capitals_goalies_2022=['Darcy Kuemper', 'Charlie Lindgren']

dict_teams_goalies_2022={'COL': team_colorado_goalies_2022, 'DAL': team_dallas_goalies_2022, 'ARI': team_coyotes_goalies_2022, 'NSH': team_predators_goalies_2022, 'STL': team_blues_goalies_2022, 'MIN': team_wild_goalies_2022, 'WPG': team_jets_goalies_2022, 'CHI': team_blackhawks_goalies_2022, 'VGK': team_knights_goalies_2022, 'VAN': team_canucks_goalies_2022, 'LAK': team_kings_goalies_2022, 'CGY': team_flames_goalies_2022, 'EDM': team_oilers_goalies_2022, 'SEA': team_kraken_goalies_2022, 'ANA': team_ducks_goalies_2022, 'SJS': team_sharks_goalies_2022, 'BOS': team_bruins_goalies_2022, 'DET': team_wings_goalies_2022, 'MTL': team_canadiens_goalies_2022, 'TOR': team_leafs_goalies_2022, 'OTT': team_senators_goalies_2022, 'TBL': team_lightning_goalies_2022, 'FLA': team_panthers_goalies_2022, 'BUF': team_sabres_goalies_2022, 'PHI': team_flyers_goalies_2022, 'CBJ': team_jackets_goalies_2022, 'NYR': team_rangers_goalies_2022, 'CAR': team_hurricanes_goalies_2022, 'NJD': team_devils_goalies_2022, 'NYI': team_islanders_goalies_2022, 'PIT': team_penguins_goalies_2022, 'WSH': team_capitals_goalies_2022}
dict_teams_2022={'COL': team_colorado_2022, 'DAL': team_dallas_2022, 'ARI': team_coyotes_2022, 'NSH': team_predators_2022, 'STL': team_blues_2022, 'MIN': team_wild_2022, 'WPG': team_jets_2022, 'CHI': team_blackhawks_2022, 'VGK': team_knights_2022, 'VAN': team_canucks_2022, 'LAK': team_kings_2022, 'CGY': team_flames_2022, 'EDM': team_oilers_2022, 'SEA': team_kraken_2022, 'ANA': team_ducks_2022, 'SJS': team_sharks_2022, 'BOS': team_bruins_2022, 'DET': team_wings_2022, 'MTL': team_canadiens_2022, 'TOR': team_leafs_2022, 'OTT': team_senators_2022, 'TBL': team_lightning_2022, 'FLA': team_panthers_2022, 'BUF': team_sabres_2022, 'PHI': team_flyers_2022, 'CBJ': team_jackets_2022, 'NYR': team_rangers_2022, 'CAR': team_hurricanes_2022, 'NJD': team_devils_2022, 'NYI': team_islanders_2022, 'PIT': team_penguins_2022, 'WSH': team_capitals_2022}

In [97]:
warnings.filterwarnings("ignore")

strengths_2022=construct_team_skater_strengths(dict_teams_2022, normalized_df.loc[normalized_df['SEASON']<=2022])
team_goalie_strengths_2022=construct_team_goalie_strengths(dict_teams_goalies_2022, smaller_goalie_df.loc[smaller_goalie_df['SEASON']<=2022])

dict_overall_team_strength_2022={}
for i in strengths_2022:
    dict_overall_team_strength_2022[i]=strengths_2022[i]+team_goalie_strengths_2022[i]

result_dict_2022 = {new_key: dict_overall_team_strength_2022[old_key] for old_key, new_key in dict_const_mapping.items()}
sorted_dict_team_strengths_2022 = dict(sorted(result_dict_2022.items(), key=lambda item: item[1], reverse=True))
elo_2022=convert_to_elo_ratings(sorted_dict_team_strengths_2022)
elo_2022

5.6010251376574285
6.144226718590383
6.1196882575966
3.621433473049577
3.6411475032367586
3.4069731136625734
0.981147924169457
2.8527209599007732
3.109890402936519
1.938929603541176
1.5329996132431463
2.357185367013194
Devon Toews
3.4
4.418981117183799
3.6415354421748947
3.645960648632953
2.615091705843841
Alexander Georgiev
Pavel Francouz
4.418981117183799
3.2692333784904033
3.878600790009103
6.053580178294381
2.865330821432099
3.1349145104266176
2.377098158246917
5.233223061247144
3.0440578514180925
2.253165018499343
Wyatt Johnson
0.3835282106601429
5.808480516580822
4.098083764446146
5.002352049566227
2.4879204605421603
3.854427801713792
1.655041953239249
Jake Oettinger
Scott Wedgewood
3.1349145104266176
4.504745155364472
3.2666317505719773
2.6950241716700214
2.775300115327843
2.7537823049072987
1.5693434753436906
0.7077815058087863
0.8157088648028524
1.2753285778057424
0.7182702965809153
1.430556558446589
Juuso Valimaki
4.507164171465463
1.9617065371960547
Janis Moser
0.22571548821

{'Florida Panthers': 1568.056790000228,
 'Los Angeles Kings': 1567.861803670351,
 'Pittsburgh Penguins': 1567.630307119107,
 'Toronto Maple Leafs': 1563.7183904121832,
 'St. Louis Blues': 1557.5776859441398,
 'Calgary Flames': 1555.7038627564605,
 'New York Rangers': 1547.3220965613746,
 'Colorado Avalanche': 1542.4070484875856,
 'Boston Bruins': 1537.8187466119334,
 'Nashville Predators': 1525.8903717785263,
 'New Jersey Devils': 1522.2767496676202,
 'Dallas Stars': 1520.5989716260399,
 'Tampa Bay Lightning': 1510.96937045997,
 'Seattle Kraken': 1510.7177125905826,
 'Columbus Blue Jackets': 1509.7256354179194,
 'Detroit Red Wings': 1508.056291861114,
 'Edmonton Oilers': 1505.6810853097456,
 'Vegas Golden Knights': 1502.4626429934237,
 'Winnipeg Jets': 1500.0360809960318,
 'Vancouver Canucks': 1496.7288600618642,
 'Carolina Hurricanes': 1492.9118963448864,
 'Ottawa Senators': 1484.0928496239785,
 'San Jose Sharks': 1480.5149710260118,
 'Washington Capitals': 1478.267857593913,
 'Buffal

In [131]:
team_panthers_2021=['Jonathan Huberdeau', 'Eetu Luostarinen', 'Sam Reinhart', 'Carter Verhaeghe', 'Anthony Duclair', 'Sam Bennett', 'Aleksander Barkov', 'Patric Hörnqvist', 'Ryan Lomberg', 'Mason Marchment', 'Maxim Mamin', 'Joe Thornton', 'Brandon Montour', 'MacKenzie Weegar', 'Radko Gudas', 'Gustav Forsling', 'Aaron Ekblad', 'Lucas Carlsson']
team_panthers_goalies_2021=['Sergei Bobrovsky', 'Spencer Knight']
team_wild_2021=['Kevin Fiala', 'Ryan Hartman', 'Kirill Kaprizov', 'Joel Eriksson Ek', 'Frédérick Gaudreau', 'Marcus Foligno', 'Mats Zuccarello', 'Jordan Greenway', 'Nick Bjugstad', 'Kyle Rau', 'Joseph Cramarossa', 'Dmitry Kulikov', 'Jonas Brodin', 'Alex Goligoski', 'Jonathon Merrill', 'Jared Spurgeon', 'Mathew Dumba', 'Jordie Benn']
team_wild_goalies_2021=['Cam Talbot']
team_senators_2021=['Alex Formenton', 'Tim Stützle', 'Brady Tkachuk', 'Chris Tierney', 'Austin Watson', 'Joshua Norris', 'Connor Brown', 'Dylan Gambrell', 'Tyler Ennis', 'Drake Batherson', 'Parker Kelly', 'Colin White', 'Artem Zub', 'Nick Holden', 'Nikita Zaitsev', 'Thomas Chabot', 'Erik Brännström', 'Victor Mete']
team_senators_goalies_2021=['Anton Forsberg', 'Matt Murray']
team_bruins_2021=['Charlie Coyle', 'Taylor Hall', 'Erik Haula', 'Jake DeBrusk', 'Tomáš Nosek', 'Craig Smith', 'Patrice Bergeron', 'David Pastrňák', 'Curtis Lazar', 'Brad Marchand', 'Nick Foligno', 'Trent Frederic', 'Brandon Carlo', 'Charlie McAvoy', 'Derek Forbort', 'Matt Grzelcyk', 'Mike Reilly', 'Connor Clifton']
team_bruins_goalies_2021=['Jeremy Swayman', 'Linus Ullmark']
team_hurricanes_2021=['Jesper Fast', 'Vincent Trocheck', 'Sebastian Aho', 'Martin Nečas', 'Jordan Staal', 'Andrei Svechnikov', 'Teuvo Teräväinen', 'Nino Niederreiter', 'Steven Lorentz', 'Jesperi Kotkaniemi', 'Jordan Martinook', 'Derek Stepan', 'Brady Skjei', 'Jaccob Slavin', 'Ian Cole', 'Brett Pesce', 'Anthony DeAngelo', 'Ethan Bear']
team_hurricanes_goalies_2021=['Frederik Andersen', 'Antti Raanta']
team_islanders_2021=['Zach Parise', 'Jean-Gabriel Pageau', 'Anders Lee', 'Anthony Beauvillier', 'Josh Bailey', 'Casey Cizikas', 'Mathew Barzal', 'Oliver Wahlstrom', 'Brock Nelson', 'Matt Martin', 'Kyle Palmieri', 'Cal Clutterbuck', 'Noah Dobson', 'Adam Pelech', 'Zdeno Chára', 'Andy Greene', 'Scott Mayfield', 'Ryan Pulock']
team_islanders_goalies_2021=['Ilya Sorokin', 'Semyon Varlamov']
team_flyers_2021=['James van Riemsdyk', 'Travis Konecny', 'Oskar Lindblom', 'Zack MacEwen', 'Cam Atkinson', 'Scott Laughton', 'Joel Farabee', 'Morgan Frost', 'Kevin Hayes', 'Patrick Brown', 'Nate Thompson', 'Sean Couturier', 'Travis Sanheim', 'Ivan Provorov', 'Keith Yandle', 'Rasmus Ristolainen', 'Nick Seeler', 'Cameron York']
team_flyers_goalies_2021=['Carter Hart', 'Martin Jones']
team_leafs_2021=['David Kämpf', 'Alexander Kerfoot', 'William Nylander', 'Michael Bunting', 'John Tavares', 'Pierre Engvall', 'Auston Matthews', 'Mitch Marner', 'Wayne Simmonds', 'Jason Spezza', 'Ilya Mikheyev', 'Ondřej Kaše', 'T.J. Brodie', 'Morgan Rielly', 'Justin Holl', 'Timothy Liljegren', 'Rasmus Sandin', 'Jake Muzzin']
team_leafs_goalies_2021=['Jack Campbell', 'Petr Mrázek']
team_canadiens_2021=['Nick Suzuki', 'Jake Evans', 'Josh Anderson', 'Cole Caufield', 'Mike Hoffman', 'Joel Armia', 'Ryan Poehling', 'Christian Dvorak', 'Brendan Gallagher', 'Laurent Dauphin', 'Jonathan Drouin', 'Paul Byron', 'Alexander Romanov', 'Jeff Petry', 'Chris Wideman', 'David Savard', 'Joel Edmundson', 'Sami Niku']
team_canadiens_goalies_2021=['Sam Montembeault', 'Jake Allen']
team_kings_2021=['Anže Kopitar', 'Trevor Moore', 'Arthur Kaliyev', 'Phillip Danault', 'Alex Iafallo', 'Adrian Kempe', 'Blake Lizotte', 'Viktor Arvidsson', 'Dustin Brown', 'Rasmus Kupari', 'Carl Grundström', 'Brendan Lemieux', 'Tobias Björnfot', 'Matt Roy', 'Olli Määttä', 'Michael Anderson', 'Alexander Edler', 'Drew Doughty']
team_kings_goalies_2021=['Jonathan Quick', 'Calvin Petersen']
team_flames_2021=['Mikael Backlund', 'Johnny Gaudreau', 'Elias Lindholm', 'Milan Lucic', 'Andrew Mangiapane', 'Matthew Tkachuk', 'Blake Coleman', 'Trevor Lewis', 'Dillon Dubé', 'Sean Monahan', 'Brett Ritchie', 'Adam Ružička', 'Rasmus Andersson', 'Chris Tanev', 'Noah Hanifin', 'Erik Gudbranson', 'Nikita Zadorov', 'Oliver Kylington']
team_flames_goalies_2021=['Jacob Markström', 'Daniel Vladař']
team_penguins_2021=['Evan Rodrigues', 'Kasperi Kapanen', 'Jeff Carter', 'Jake Guentzel', 'Danton Heinen', 'Sidney Crosby', 'Brian Boyle', 'Teddy Blueger', 'Brock McGinn', 'Bryan Rust', 'Evgeni Malkin', 'Jason Zucker', 'John Marino', 'Kris Letang', 'Chad Ruhwedel', 'Brian Dumoulin', 'Mike Matheson', 'Marcus Pettersson']
team_penguins_goalies_2021=['Tristan Jarry', 'Casey DeSmith']
team_oilers_2021=['Warren Foegele', 'Kailer Yamamoto', 'Leon Draisaitl', 'Connor McDavid', 'Zach Hyman', 'Derek Ryan', 'Ryan McLeod', 'Jesse Puljujärvi', 'Ryan Nugent-Hopkins', 'Zack Kassian', 'Devin Shore', 'Evander Kane', 'Evan Bouchard', 'Cody Ceci', 'Tyson Barrie', 'Darnell Nurse', 'Duncan Keith', 'Kris Russell']
team_oilers_goalies_2021=['Mikko Koskinen', 'Mike Smith']
team_sabres_2021=['Rasmus Asplund', 'Jeff Skinner', 'Dylan Cozens', 'Tage Thompson', 'Kyle Okposo', 'Victor Olofsson', 'Cody Eakin', 'Vinnie Hinostroza', 'Anders Bjork', 'Zemgus Girgensons', 'John Hayden', 'Alex Tuch', 'Rasmus Dahlin', 'Jacob Bryson', 'Mark Pysyk', 'Henri Jokiharju', 'Mattias Samuelsson', 'Colin Miller']
team_sabres_goalies_2021=['Craig Anderson', 'Dustin Tokarski']
team_capitals_2021=['Evgeny Kuznetsov', 'Tom Wilson', 'Alex Ovechkin', 'Garnet Hathaway', 'Lars Eller', 'Conor Sheary', 'Connor McMichael', 'Nic Dowd', 'Carl Hagelin', 'Nicklas Bäckström', 'T.J. Oshie', 'Anthony Mantha', 'Martin Fehérváry', 'John Carlson', 'Nick Jensen', 'Dmitry Orlov', 'Justin Schultz', 'Trevor van Riemsdyk']
team_capitals_goalies_2021=['Ilya Samsonov', 'Vítek Vaněček']
team_knights_2021=['Chandler Stephenson', 'Evgenii Dadonov', 'Nicolas Roy', 'Keegan Kolesar', 'Jonathan Marchessault', 'Mattias Janmark', 'William Karlsson', 'William Carrier', 'Reilly Smith', 'Brett Howden', 'Max Pacioretty', 'Mark Stone', 'Alex Pietrangelo', 'Shea Theodore', 'Brayden McNabb', 'Dylan Coghlan', 'Zach Whitecloud', 'Ben Hutton']
team_knights_goalies_2021=['Robin Lehner', 'Laurent Brossoit']
team_devils_2021=['Michael McLeod', 'Jesper Bratt', 'Yegor Sharangovich', 'Tomáš Tatar', 'Andreas Johnsson', 'Nico Hischier', 'Pavel Zacha', 'Jimmy Vesey', 'Janne Kuokkanen', 'Jesper Boqvist', 'Jack Hughes', 'A.J. Greer', 'Damon Severson', 'P.K. Subban', 'Ryan Graves', 'Jonas Siegenthaler', 'Ty Smith', 'Dougie Hamilton']
team_devils_goalies_2021=['Mackenzie Blackwood', 'Jonathan Bernier']
team_canucks_2021=['J.T. Miller', 'Elias Pettersson', 'Conor Garland', 'Juho Lammikko', 'Brock Boeser', 'Bo Horvat', 'Tanner Pearson', 'Alex Chiasson', 'Jason Dickinson', 'Nils Höglander', 'Matthew Highmore', 'Justin Dowling', 'Tyler Myers', 'Oliver Ekman-Larsson', 'Quinn Hughes', 'Luke Schenn', 'Brad Hunt', 'Kyle Burroughs']
team_canucks_goalies_2021=['Thatcher Demko', 'Jaroslav Halák']
team_sharks_2021=['Tomáš Hertl', 'Nick Bonino', 'Logan Couture', 'Timo Meier', 'Alexander Barabanov', 'Matthew Nieto', 'Noah Gregor', 'Rūdolfs Balcers', 'Jonah Gadjovich', 'Jeffrey Viel', 'Lane Pederson', 'Kevin Labanc', 'Brent Burns', 'Marc-Édouard Vlasic', 'Mario Ferraro', 'Erik Karlsson', 'Nicolas Meloche', 'Jaycob Megna']
team_sharks_goalies_2021=['James Reimer', 'Adin Hill']
team_blues_2021=['Ivan Barbashev', "Ryan O'Reilly", 'Brandon Saad', 'Vladimir Tarasenko', 'Jordan Kyrou', 'Pavel Buchnevich', 'Robert Thomas', 'David Perron', 'Brayden Schenn', 'Tyler Bozak', 'Klim Kostin', 'Logan Brown', 'Colton Parayko', 'Justin Faulk', 'Robert Bortuzzo', 'Marco Scandella', 'Torey Krug', 'Niko Mikkola']
team_blues_goalies_2021=['Ville Husso', 'Jordan Binnington']
team_wings_2021=['Pius Suter', 'Sam Gagner', 'Michael Rasmussen', 'Adam Erne', 'Filip Zadina', 'Dylan Larkin', 'Tyler Bertuzzi', 'Joe Veleno', 'Robby Fabbri', 'Givani Smith', 'Mitchell Stephens', 'Carter Rowney', 'Filip Hronek', 'Marc Staal', 'Gustav Lindström', 'Danny DeKeyser', 'Jordan Oesterle', 'Dan Renouf']
team_wings_goalies_2021=['Alex Nedeljkovic', 'Thomas Greiss']
team_jackets_2021=['Gustav Nyquist', 'Jack Roslovic', 'Oliver Bjorkstrand', 'Jakub Voráček', 'Sean Kuraly', 'Eric Robinson', 'Boone Jenner', 'Patrik Laine', 'Emil Bemström', 'Alexandre Texier', 'Brendan Gaunce', 'Kevin Stenlund', 'Andrew Peeke', 'Vladislav Gavrikov', 'Zach Werenski', 'Jake Bean', 'Adam Boqvist', 'Gavin Bayreuther']
team_jackets_goalies_2021=['Elvis Merzļikins', 'Joonas Korpisalo']
team_coyotes_2021=['Phil Kessel', 'Travis Boyd', 'Loui Eriksson', 'Clayton Keller', 'Lawson Crouse', 'Nick Schmaltz', 'Alex Galchenyuk', 'Barrett Hayton', 'Christian Fischer', 'Antoine Roussel', 'Andrew Ladd', "Liam O'Brien", 'Shayne Gostisbehere', 'Anton Strålman', 'Jakob Chychrun', 'Kyle Capobianco', 'Victor Söderström', 'Conor Timmins']
team_coyotes_goalies_2021=['Harri Säteri', 'Carter Hutton']
team_blackhawks_2021=['Alex DeBrincat', 'Patrick Kane', 'Dominik Kubalík', 'Jonathan Toews', 'Kirby Dach', 'Dylan Strome', 'Philipp Kurashev', 'Mackenzie Entwistle', 'Henrik Borgström', 'Reese Johnson', 'Jujhar Khaira', 'Tyler Johnson', 'Seth Jones', 'Jake McCabe', 'Calvin de Haan', 'Erik Gustafsson', 'Connor Murphy', 'Riley Stillman']
team_blackhawks_goalies_2021=['Kevin Lankinen', 'Collin Delia']
team_jets_2021=['Pierre-Luc Dubois', 'Kyle Connor', 'Adam Lowry', 'Jansen Harkins', 'Dominic Toninato', 'Evgeny Svechnikov', 'Paul Stastny', 'Mark Scheifele', 'Blake Wheeler', 'Nikolaj Ehlers', 'Kristian Vesalainen', 'Austin Poganski', 'Brenden Dillon', 'Josh Morrissey', 'Neal Pionk', 'Nate Schmidt', 'Dylan DeMelo', 'Logan Stanley']
team_jets_goalies_2021=['Connor Hellebuyck', 'Eric Comrie']
team_lightning_2021=['Alex Killorn', 'Corey Perry', 'Pat Maroon', 'Steven Stamkos', 'Pierre-Édouard Bellemare', 'Ross Colton', 'Ondřej Palát', 'Anthony Cirelli', 'Brayden Point', 'Nikita Kucherov', 'Remi Elie', 'Victor Hedman', 'Mikhail Sergachev', 'Jan Rutta', 'Ryan McDonagh', 'Callan Foote', 'Erik Černák', 'Zach Bogosian']
team_lightning_goalies_2021=['Andrei Vasilevskiy', 'Brian Elliott']
team_dallas_2021=['Jamie Benn', 'Luke Glendening', 'Joe Pavelski', 'Tyler Seguin', 'Roope Hintz', 'Radek Faksa', 'Michael Raffl', 'Jason Robertson', 'Denis Gurianov', 'Alexander Radulov', 'Joel Kiviranta', 'Tanner Kero', 'Ryan Suter', 'Jani Hakanpää', 'Esa Lindell', 'John Klingberg', 'Miro Heiskanen', 'Joel Hanley']
team_dallas_goalies_2021=['Jake Oettinger', 'Braden Holtby']
team_predators_2021=['Luke Kunin', 'Tanner Jeannot', 'Mikael Granlund', 'Yakov Trenin', 'Ryan Johansen', 'Colton Sissons', 'Matt Duchene', 'Eeli Tolvanen', 'Filip Forsberg', 'Nick Cousins', 'Michael McCarron', 'Matt Luff', 'Roman Josi', 'Alexandre Carrier', 'Mattias Ekholm', 'Dante Fabbro', 'Matt Benning', 'Mark Borowiecki']
team_predators_goalies_2021=['Juuse Saros', 'David Rittich']
team_ducks_2021=['Isac Lundeström', 'Troy Terry', 'Trevor Zegras', 'Derek Grant', 'Sam Steel', 'Sonny Milano', 'Sam Carrick', 'Adam Henrique', 'Ryan Getzlaf', 'Jakob Silfverberg', 'Max Comtois', 'Buddy Robinson', 'Kevin Shattenkirk', 'Jamie Drysdale', 'Cam Fowler', 'Simon Benoit', 'Josh Mahura', 'Greg Pateryn']
team_ducks_goalies_2021=['John Gibson', 'Anthony Stolarz']
team_rangers_2021=['Chris Kreider', 'Mika Zibanejad', 'Barclay Goodrow', 'Alexis Lafrenière', 'Dryden Hunt', 'Artemi Panarin', 'Ryan Strome', 'Ryan Reaves', 'Filip Chytil', 'Kevin Rooney', 'Julien Gauthier', 'Kaapo Kakko', "K'Andre Miller", 'Jacob Trouba', 'Adam Fox', 'Ryan Lindgren', 'Patrik Nemeth', 'Libor Hájek']
team_rangers_goalies_2021=['Igor Shesterkin', 'Alexandar Georgiev']
team_kraken_2021=['Alexander Wennberg', 'Jordan Eberle', 'Joonas Donskoi', 'Ryan Donato', 'Yanni Gourde', 'Jared McCann', 'Morgan Geekie', 'Riley Sheahan', 'Jaden Schwartz', 'Brandon Tanev', 'Kole Lind', 'Max McCormick', 'Adam Larsson', 'Vince Dunn', 'Jamie Oleksiak', 'Carson Soucy', 'William Borgen', 'Haydn Fleury']
team_kraken_goalies_2021=['Philipp Grubauer', 'Chris Driedger']
team_colorado_2021=["Logan O'Connor", 'André Burakovsky', 'Mikko Rantanen', 'Nazem Kadri', 'Alex Newhook', 'J.T. Compher', 'Darren Helm', 'Nathan MacKinnon', 'Valeri Nichushkin', 'Gabriel Landeskog', 'Jayson Megna', 'Mikhail Maltsev', 'Erik Johnson', 'Cale Makar', 'Jack Johnson', 'Samuel Girard', 'Devon Toews', 'Kurtis MacDermid']
team_colorado_goalies_2021=['Darcy Kuemper', 'Pavel Francouz']

dict_teams_goalies_2021={'COL': team_colorado_goalies_2021, 'DAL': team_dallas_goalies_2021, 'ARI': team_coyotes_goalies_2021, 'NSH': team_predators_goalies_2021, 'STL': team_blues_goalies_2021, 'MIN': team_wild_goalies_2021, 'WPG': team_jets_goalies_2021, 'CHI': team_blackhawks_goalies_2021, 'VGK': team_knights_goalies_2021, 'VAN': team_canucks_goalies_2021, 'LAK': team_kings_goalies_2021, 'CGY': team_flames_goalies_2021, 'EDM': team_oilers_goalies_2021, 'SEA': team_kraken_goalies_2021, 'ANA': team_ducks_goalies_2021, 'SJS': team_sharks_goalies_2021, 'BOS': team_bruins_goalies_2021, 'DET': team_wings_goalies_2021, 'MTL': team_canadiens_goalies_2021, 'TOR': team_leafs_goalies_2021, 'OTT': team_senators_goalies_2021, 'TBL': team_lightning_goalies_2021, 'FLA': team_panthers_goalies_2021, 'BUF': team_sabres_goalies_2021, 'PHI': team_flyers_goalies_2021, 'CBJ': team_jackets_goalies_2021, 'NYR': team_rangers_goalies_2021, 'CAR': team_hurricanes_goalies_2021, 'NJD': team_devils_goalies_2021, 'NYI': team_islanders_goalies_2021, 'PIT': team_penguins_goalies_2021, 'WSH': team_capitals_goalies_2021}
dict_teams_2021={'COL': team_colorado_2021, 'DAL': team_dallas_2021, 'ARI': team_coyotes_2021, 'NSH': team_predators_2021, 'STL': team_blues_2021, 'MIN': team_wild_2021, 'WPG': team_jets_2021, 'CHI': team_blackhawks_2021, 'VGK': team_knights_2021, 'VAN': team_canucks_2021, 'LAK': team_kings_2021, 'CGY': team_flames_2021, 'EDM': team_oilers_2021, 'SEA': team_kraken_2021, 'ANA': team_ducks_2021, 'SJS': team_sharks_2021, 'BOS': team_bruins_2021, 'DET': team_wings_2021, 'MTL': team_canadiens_2021, 'TOR': team_leafs_2021, 'OTT': team_senators_2021, 'TBL': team_lightning_2021, 'FLA': team_panthers_2021, 'BUF': team_sabres_2021, 'PHI': team_flyers_2021, 'CBJ': team_jackets_2021, 'NYR': team_rangers_2021, 'CAR': team_hurricanes_2021, 'NJD': team_devils_2021, 'NYI': team_islanders_2021, 'PIT': team_penguins_2021, 'WSH': team_capitals_2021}

In [129]:
warnings.filterwarnings("ignore")

strengths_2021=construct_team_skater_strengths(dict_teams_2021, normalized_df.loc[normalized_df['SEASON']<=2021])
team_goalie_strengths_2021=construct_team_goalie_strengths(dict_teams_goalies_2021, smaller_goalie_df.loc[smaller_goalie_df['SEASON']<=2021])

dict_overall_team_strength_2021={}
for i in strengths_2021:
    dict_overall_team_strength_2021[i]=strengths_2021[i]+team_goalie_strengths_2021[i]

result_dict_2021 = {new_key: dict_overall_team_strength_2021[old_key] for old_key, new_key in dict_const_mapping.items()}
sorted_dict_team_strengths_2021 = dict(sorted(result_dict_2021.items(), key=lambda item: item[1], reverse=True))
elo_2021=convert_to_elo_ratings(sorted_dict_team_strengths_2021)
elo_2021

0.7141343130785529
3.1634183195058676
4.097056580755983
2.774457984014522
Alex Newhook
2.6316336231687023
1.607356985650456
6.742003907348101
3.3184539866085037
4.208299111126187
0.26603584226223276
Mikhail Maltsev
2.565945813205287
4.40392166273926
2.920409259521903
4.70272281428479
3.1615539780898922
2.1008863375800746
2.2391829539736667
0.755479630000417
2.561208281319365
3.190010160734173
2.180774046798523
2.323812875385341
1.5489974609646082
0.5919003078507608
3.556338756714768
2.231778464610364
0.5951521632662475
0.10378139745783456
3.6979957332444506
0.5554702495201534
3.783866536859344
4.745111340828164
4.472461574727914
0.8080920632648119
2.757590338908574
1.7842310861052335
1.2706657771178511
3.6800805037178845
2.591954922842302
3.208682414854743
1.3445764084980159
2.109078150941132
1.9666154146038908
1.4256366435674364
1.0522535015947856
0.3484426296944709
2.6903948236623854
3.6289833953962907
3.8336058970510334
1.3898598779218683
Victor Söderström
1.1533604887983706
2.68458

{'Pittsburgh Penguins': 1607.9335581373457,
 'Carolina Hurricanes': 1599.198620028416,
 'Tampa Bay Lightning': 1580.6670871477377,
 'Boston Bruins': 1558.2175517215894,
 'Edmonton Oilers': 1551.65894854592,
 'Vegas Golden Knights': 1546.2183307379987,
 'New York Rangers': 1544.6533127618593,
 'Columbus Blue Jackets': 1540.3837170076827,
 'Philadelphia Flyers': 1538.501123647319,
 'Winnipeg Jets': 1535.0139236534424,
 'Toronto Maple Leafs': 1532.363363082539,
 'Colorado Avalanche': 1529.1983965702887,
 'New York Islanders': 1520.153564420376,
 'Vancouver Canucks': 1509.289347390251,
 'Calgary Flames': 1509.066547175526,
 'Nashville Predators': 1507.1441512429278,
 'Washington Capitals': 1492.7416594647937,
 'St. Louis Blues': 1486.708723829913,
 'Los Angeles Kings': 1481.263031172729,
 'New Jersey Devils': 1479.0903227419576,
 'Florida Panthers': 1464.2144463336604,
 'Minnesota Wild': 1456.1874906659184,
 'Dallas Stars': 1448.5713531085048,
 'Montreal Canadiens': 1447.38360183607,
 'Ott

In [106]:
team_wings_2019=['Luke Glendening', 'Andreas Athanasiou', 'Dylan Larkin', 'Tyler Bertuzzi', 'Frans Nielsen', 'Justin Abdelkader', 'Anthony Mantha', 'Thomas Vanek', 'Darren Helm', 'Jacob de La Rose', 'Martin Frk', 'Wade Megan', 'Niklas Kronwall', 'Danny DeKeyser', 'Jonathan Ericsson', 'Trevor Daley', 'Mike Green', 'Luke Witkowski']
team_wings_goalies_2019=['Jimmy Howard', 'Jonathan Bernier']
team_bruins_2019=['David Krejčí', 'Brad Marchand', 'Danton Heinen', 'Chris Wagner', 'Noel Acciari', 'Sean Kuraly', 'David Backes', 'Joakim Nordström', 'Jake DeBrusk', 'David Pastrňák', 'Patrice Bergeron', 'Jakob Forsbacka Karlsson', 'Brandon Carlo', 'Matt Grzelcyk', 'Torey Krug', 'Zdeno Chára', 'John Moore', 'Charlie McAvoy']
team_bruins_goalies_2019=['Tuukka Rask', 'Jaroslav Halák']
team_colorado_2019=['Matt Calvert', 'Nathan MacKinnon', 'Carl Söderberg', 'Alexander Kerfoot', 'Mikko Rantanen', 'Gabriel Landeskog', 'Tyson Jost', 'J.T. Compher', 'Colin Wilson', 'Sven Andrighetto', 'Matthew Nieto', 'Gabriel Bourque', 'Samuel Girard', 'Erik Johnson', 'Tyson Barrie', 'Patrik Nemeth', 'Ian Cole', 'Nikita Zadorov']
team_colorado_goalies_2019=['Semyon Varlamov', 'Philipp Grubauer']
team_hurricanes_2019=['Sebastian Aho', 'Jordan Martinook', 'Brock McGinn', 'Teuvo Teräväinen', 'Justin Williams', 'Lucas Wallmark', 'Warren Foegele', 'Micheal Ferland', 'Jordan Staal', 'Greg McKegg', 'Janne Kuokkanen', 'Martin Nečas', 'Justin Faulk', 'Dougie Hamilton', 'Jaccob Slavin', 'Trevor van Riemsdyk', 'Calvin de Haan', 'Brett Pesce']
team_hurricanes_goalies_2019=['Petr Mrázek', 'Curtis McElhinney']
team_canadiens_2019=['Max Domi', 'Brendan Gallagher', 'Artturi Lehkonen', 'Phillip Danault', 'Jonathan Drouin', 'Tomáš Tatar', 'Andrew Shaw', 'Joel Armia', 'Paul Byron', 'Nicolas Deslauriers', 'Matthew Peca', 'Michael Chaput', 'Jeff Petry', 'Jordie Benn', 'Victor Mete', 'Shea Weber', 'Brett Kulak', 'Mike Reilly']
team_canadiens_goalies_2019=['Carey Price', 'Antti Niemi']
team_kings_2019=['Alex Iafallo', 'Tyler Toffoli', 'Adrian Kempe', 'Anže Kopitar', 'Jeff Carter', 'Dustin Brown', 'Kyle Clifford', 'Ilya Kovalchuk', 'Trevor Lewis', 'Michael Amadio', 'Jonny Brodzinski', 'Nikita Scherbak', 'Drew Doughty', 'Derek Forbort', 'Dion Phaneuf', 'Alec Martinez', 'Paul Ladue', 'Kurtis MacDermid']
team_kings_goalies_2019=['Jonathan Quick', 'Jack Campbell']
team_jackets_2019=['Josh Anderson', 'Pierre-Luc Dubois', 'Cam Atkinson', 'Artemi Panarin', 'Riley Nash', 'Oliver Bjorkstrand', 'Boone Jenner', 'Alexander Wennberg', 'Nick Foligno', 'Brandon Dubinsky', 'Lukáš Sedlák', 'Markus Hännikäinen', 'David Savard', 'Zach Werenski', 'Markus Nutivaara', 'Seth Jones', 'Scott Harrington', 'Ryan Murray']
team_jackets_goalies_2019=['Sergei Bobrovsky', 'Joonas Korpisalo']
team_rangers_2019=['Mika Zibanejad', 'Jimmy Vesey', 'Chris Kreider', 'Vladislav Namestnikov', 'Filip Chytil', 'Jesper Fast', 'Pavel Buchnevich', 'Boo Nieves', 'Lias Andersson', 'Vinni Lettieri', 'Connor Brickley', 'Steven Fogarty', 'Marc Staal', 'Brady Skjei', 'Neal Pionk', 'Kevin Shattenkirk', 'Brendan Smith', 'Anthony DeAngelo']
team_rangers_goalies_2019=['Henrik Lundqvist', 'Alexandar Georgiev']
team_flames_2019=['Johnny Gaudreau', 'Elias Lindholm', 'Derek Ryan', 'Matthew Tkachuk', 'Mark Jankowski', 'Sean Monahan', 'Mikael Backlund', 'Garnet Hathaway', 'Sam Bennett', 'Michael Frolík', 'James Neal', 'Austin Czarnik', 'Noah Hanifin', 'Rasmus Andersson', 'T.J. Brodie', 'Mark Giordano', 'Travis Hamonic', 'Oliver Kylington']
team_flames_goalies_2019=['David Rittich', 'Mike Smith']
team_blackhawks_2019=['Alex DeBrincat', 'Jonathan Toews', 'Patrick Kane', 'Brandon Saad', 'Artem Anisimov', 'Marcus Krüger', 'David Kämpf', 'Chris Kunitz', 'John Hayden', 'Dylan Sikura', 'Andreas Martinsen', 'Duncan Keith', 'Erik Gustafsson', 'Brent Seabrook', 'Connor Murphy', 'Gustav Forsling', 'Carl Dahlström', 'Brandon Davidson']
team_blackhawks_goalies_2019=['Corey Crawford', 'Cam Ward']
team_coyotes_2019=['Clayton Keller', 'Nick Cousins', 'Lawson Crouse', 'Richard Pánik', 'Alex Galchenyuk', 'Vinnie Hinostroza', 'Derek Stepan', 'Christian Fischer', 'Josh Archibald', 'Brad Richardson', 'Mario Kempe', 'Michael Grabner', 'Niklas Hjalmarsson', 'Oliver Ekman-Larsson', 'Alex Goligoski', 'Jordan Oesterle', 'Jakob Chychrun', 'Kevin Connauton']
team_coyotes_goalies_2019=['Darcy Kuemper', 'Adin Hill']
team_predators_2019=['Nick Bonino', 'Ryan Johansen', 'Calle Järnkrok', 'Craig Smith', 'Colton Sissons', 'Filip Forsberg', 'Viktor Arvidsson', 'Frédérick Gaudreau', 'Kyle Turris', 'Rocco Grimaldi', 'Miikka Salomäki', 'Austin Watson', 'Ryan Ellis', 'Roman Josi', 'Mattias Ekholm', 'P.K. Subban', 'Yannick Weber', 'Dan Hamhuis']
team_predators_goalies_2019=['Pekka Rinne', 'Juuse Saros']
team_penguins_2019=['Jake Guentzel', 'Phil Kessel', 'Sidney Crosby', 'Bryan Rust', 'Matt Cullen', 'Dominik Simon', 'Patric Hörnqvist', 'Evgeni Malkin', 'Garrett Wilson', 'Zach Aston-Reese', 'Jack Johnson', 'Brian Dumoulin', 'Kris Letang', 'Olli Määttä', 'Justin Schultz', 'Chad Ruhwedel', 'Zach Trotman']
team_penguins_goalies_2019=['Matt Murray', 'Casey DeSmith']
team_capitals_2019=['Jakub Vrána', 'Brett Connolly', 'Lars Eller', 'Alex Ovechkin', 'Nicklas Bäckström', 'André Burakovsky', 'Evgeny Kuznetsov', 'T.J. Oshie', 'Nic Dowd', 'Chandler Stephenson', 'Tom Wilson', 'Devante Smith-Pelly', 'Dmitry Orlov', 'John Carlson', 'Matt Niskanen', 'Michal Kempný', 'Brooks Orpik', 'Christian Djoos']
team_capitals_goalies_2019=['Braden Holtby', 'Pheonix Copley']
team_canucks_2019=['Bo Horvat', 'Loui Eriksson', 'Markus Granlund', 'Tyler Motte', 'Jake Virtanen', 'Brock Boeser', 'Antoine Roussel', 'Nikolay Goldobin', 'Jay Beagle', 'Adam Gaudette', 'Tim Schaller', 'Sven Baertschi', 'Troy Stecher', 'Ben Hutton', 'Derrick Pouliot', 'Alexander Edler', 'Chris Tanev', 'Alex Biega']
team_canucks_goalies_2019=['Jacob Markström', 'Thatcher Demko']
team_islanders_2019=['Josh Bailey', 'Mathew Barzal', 'Leo Komarov', 'Anders Lee', 'Brock Nelson', 'Anthony Beauvillier', 'Jordan Eberle', 'Casey Cizikas', 'Cal Clutterbuck', 'Valtteri Filppula', 'Matt Martin', 'Tom Kühnhackl', 'Nick Leddy', 'Ryan Pulock', 'Scott Mayfield', 'Adam Pelech', 'Johnny Boychuk', 'Thomas Hickey']
team_islanders_goalies_2019=['Robin Lehner', 'Thomas Greiss']
team_flyers_2019=['Claude Giroux', 'Travis Konecny', 'Scott Laughton', 'Oskar Lindblom', 'Sean Couturier', 'Jakub Voráček', 'Nolan Patrick', 'Michael Raffl', 'James van Riemsdyk', 'Phil Varone', 'Jori Lehterä', 'Corban Knight', 'Robert Hägg', 'Ivan Provorov', 'Travis Sanheim', 'Shayne Gostisbehere', 'Radko Gudas', 'Andrew MacDonald']
team_flyers_goalies_2019=['Brian Elliott', 'Michal Neuvirth']
team_blues_2019=["Ryan O'Reilly", 'Ivan Barbashev', 'Vladimir Tarasenko', 'Pat Maroon', 'Oskar Sundqvist', 'Tyler Bozak', 'Brayden Schenn', 'Jaden Schwartz', 'Alexander Steen', 'Zach Sanford', 'David Perron', 'Sammy Blais', 'Colton Parayko', 'Jay Bouwmeester', 'Vince Dunn', 'Alex Pietrangelo', 'Joel Edmundson', 'Robert Bortuzzo']
team_blues_goalies_2019=['Jake Allen', 'Jordan Binnington']
team_panthers_2019=['Aleksander Barkov', 'Evgenii Dadonov', 'Mike Hoffman', 'Jonathan Huberdeau', 'Frank Vatrano', 'Troy Brouwer', 'Colton Sceviour', 'Vincent Trocheck', 'Henrik Borgström', 'Denis Malgin', 'Dryden Hunt', 'Jamie McGinn', 'Aaron Ekblad', 'Keith Yandle', 'Mike Matheson', 'Mark Pysyk', 'MacKenzie Weegar', 'Ian McCoshen']
team_panthers_goalies_2019=['Roberto Luongo', 'James Reimer']
team_wild_2019=['Marcus Foligno', 'Jordan Greenway', 'Eric Staal', 'Jason Zucker', 'Zach Parise', 'Eric Fehr', 'Joel Eriksson Ek', 'J.T. Brown', 'Luke Kunin', 'Mikko Koivu', 'Matt Read', 'Kyle Rau', 'Jonas Brodin', 'Jared Spurgeon', 'Ryan Suter', 'Greg Pateryn', 'Nick Seeler', 'Mathew Dumba']
team_wild_goalies_2019=['Devan Dubnyk', 'Alex Stalock']
team_knights_2019=['William Karlsson', 'Jonathan Marchessault', 'Ryan Reaves', 'Cody Eakin', 'Pierre-Édouard Bellemare', 'Reilly Smith', 'Alex Tuch', 'Ryan Carpenter', 'Tomáš Nosek', 'Max Pacioretty', 'William Carrier', 'Paul Stastny', 'Brayden McNabb', 'Shea Theodore', 'Deryk Engelland', 'Colin Miller', 'Nick Holden', 'Nate Schmidt']
team_knights_goalies_2019=['Marc-André Fleury', 'Malcolm Subban']
team_dallas_2019=['Tyler Seguin', 'Radek Faksa', 'Mattias Janmark', 'Jamie Benn', 'Blake Comeau', 'Jason Spezza', 'Alexander Radulov', 'Jason Dickinson', 'Valeri Nichushkin', 'Brett Ritchie', 'Tyler Pitlick', 'Denis Gurianov', 'Esa Lindell', 'Roman Polák', 'John Klingberg', 'Taylor Fedun', 'Julius Honka', 'Joel Hanley']
team_dallas_goalies_2019=['Ben Bishop', 'Anton Khudobin']
team_oilers_2019=['Leon Draisaitl', 'Ryan Nugent-Hopkins', 'Zack Kassian', 'Milan Lucic', 'Connor McDavid', 'Alex Chiasson', 'Kyle Brodziak', 'Tobias Rieder', 'Jujhar Khaira', 'Ty Rattie', 'Jesse Puljujärvi', 'Kailer Yamamoto', 'Adam Larsson', 'Darnell Nurse', 'Kris Russell', 'Matt Benning', 'Oscar Klefbom', 'Kevin Gravel']
team_oilers_goalies_2019=['Mikko Koskinen']
team_sabres_2019=['Sam Reinhart', 'Jeff Skinner', 'Kyle Okposo', 'Conor Sheary', 'Jack Eichel', 'Casey Mittelstadt', 'Evan Rodrigues', 'Johan Larsson', 'Jason Pominville', 'Zemgus Girgensons', 'Vladimír Sobotka', 'Tage Thompson', 'Rasmus Ristolainen', 'Zach Bogosian', 'Marco Scandella', 'Jake McCabe', 'Casey Nelson', 'Matt Hunwick']
team_sabres_goalies_2019=['Carter Hutton', 'Linus Ullmark']
team_senators_2019=['Chris Tierney', 'Magnus Pääjärvi', 'Bobby Ryan', 'Mikkel Bødker', 'Colin White', 'Zack Smith', 'Jean-Gabriel Pageau', 'Tom Pyatt', 'Nick Paul', 'Max McCormick', 'Alex Formenton', 'Jack Rodewald', 'Dylan DeMelo', 'Cody Ceci', 'Thomas Chabot', 'Christián Jaroš', 'Mark Borowiecki', 'Ben Harpur']
team_senators_goalies_2019=['Craig Anderson', 'Mike Condon']
team_devils_2019=['Travis Zajac', 'Blake Coleman', 'Kyle Palmieri', 'Nico Hischier', 'Miles Wood', 'Pavel Zacha', 'Drew Stafford', 'Jesper Bratt', 'Stefan Noesen', 'Kevin Rooney', 'Taylor Hall', 'Kurtis Gabriel', 'Andy Greene', 'Damon Severson', 'Will Butcher', 'Mirco Müller', 'Sami Vatanen', 'Steven Santini']
team_devils_goalies_2019=['Keith Kinkaid', 'Cory Schneider']
team_sharks_2019=['Barclay Goodrow', 'Kevin Labanc', 'Logan Couture', 'Joonas Donskoi', 'Marcus Sorensen', 'Melker Karlsson', 'Timo Meier', 'Tomáš Hertl', 'Evander Kane', 'Joe Pavelski', 'Joe Thornton', 'Dylan Gambrell', 'Brent Burns', 'Brenden Dillon', 'Justin Braun', 'Marc-Édouard Vlasic', 'Erik Karlsson', 'Joakim Ryan']
team_sharks_goalies_2019=['Martin Jones', 'Aaron Dell']
team_leafs_2019=['Connor Brown', 'Patrick Marleau', 'Mitch Marner', 'John Tavares', 'Kasperi Kapanen', 'Andreas Johnsson', 'Nazem Kadri', 'Zach Hyman', 'Frederik Gauthier', 'Auston Matthews', 'William Nylander', 'Tyler Ennis', 'Morgan Rielly', 'Ron Hainsey', 'Nikita Zaitsev', 'Travis Dermott', 'Jake Gardiner', 'Martin Marinčin']
team_leafs_goalies_2019=['Frederik Andersen', 'Garret Sparks']
team_jets_2019=['Kyle Connor', 'Patrik Laine', 'Bryan Little', 'Mathieu Perreault', 'Mark Scheifele', 'Blake Wheeler', 'Brandon Tanev', 'Adam Lowry', 'Jack Roslovic', 'Andrew Copp', 'Nikolaj Ehlers', 'Jacob Trouba', 'Tyler Myers', 'Ben Chiarot', 'Josh Morrissey', 'Dmitry Kulikov', 'Dustin Byfuglien', 'Joseph Morrow']
team_jets_goalies_2019=['Connor Hellebuyck', 'Laurent Brossoit']
team_lightning_2019=['Anthony Cirelli', 'Alex Killorn', 'Nikita Kucherov', 'Steven Stamkos', 'Yanni Gourde', 'Tyler Johnson', 'Cédric Paquette', 'Brayden Point', 'J.T. Miller', 'Adam Erne', 'Ondřej Palát', 'Ryan Callahan', 'Ryan McDonagh', 'Mikhail Sergachev', 'Braydon Coburn', 'Victor Hedman', 'Dan Girardi', 'Anton Strålman']
team_lightning_goalies_2019=['Andrei Vasilevskiy', 'Louis Domingue']
team_ducks_2019=['Adam Henrique', 'Jakob Silfverberg', 'Rickard Rakell', 'Ryan Getzlaf', 'Carter Rowney', 'Ryan Kesler', 'Nick Ritchie', 'Troy Terry', 'Corey Perry', 'Ondřej Kaše', 'Ben Street', 'Kalle Kossila', 'Hampus Lindholm', 'Josh Manson', 'Cam Fowler', 'Jacob Larsson', 'Jaycob Megna', 'Andy Welinski']
team_ducks_goalies_2019=['John Gibson', 'Ryan Miller']

dict_teams_goalies_2019={'COL': team_colorado_goalies_2019, 'DAL': team_dallas_goalies_2019, 'ARI': team_coyotes_goalies_2019, 'NSH': team_predators_goalies_2019, 'STL': team_blues_goalies_2019, 'MIN': team_wild_goalies_2019, 'WPG': team_jets_goalies_2019, 'CHI': team_blackhawks_goalies_2019, 'VGK': team_knights_goalies_2019, 'VAN': team_canucks_goalies_2019, 'LAK': team_kings_goalies_2019, 'CGY': team_flames_goalies_2019, 'EDM': team_oilers_goalies_2019, 'ANA': team_ducks_goalies_2019, 'SJS': team_sharks_goalies_2019, 'BOS': team_bruins_goalies_2019, 'DET': team_wings_goalies_2019, 'MTL': team_canadiens_goalies_2019, 'TOR': team_leafs_goalies_2019, 'OTT': team_senators_goalies_2019, 'TBL': team_lightning_goalies_2019, 'FLA': team_panthers_goalies_2019, 'BUF': team_sabres_goalies_2019, 'PHI': team_flyers_goalies_2019, 'CBJ': team_jackets_goalies_2019, 'NYR': team_rangers_goalies_2019, 'CAR': team_hurricanes_goalies_2019, 'NJD': team_devils_goalies_2019, 'NYI': team_islanders_goalies_2019, 'PIT': team_penguins_goalies_2019, 'WSH': team_capitals_goalies_2019}
dict_teams_2019={'COL': team_colorado_2019, 'DAL': team_dallas_2019, 'ARI': team_coyotes_2019, 'NSH': team_predators_2019, 'STL': team_blues_2019, 'MIN': team_wild_2019, 'WPG': team_jets_2019, 'CHI': team_blackhawks_2019, 'VGK': team_knights_2019, 'VAN': team_canucks_2019, 'LAK': team_kings_2019, 'CGY': team_flames_2019, 'EDM': team_oilers_2019, 'ANA': team_ducks_2019, 'SJS': team_sharks_2019, 'BOS': team_bruins_2019, 'DET': team_wings_2019, 'MTL': team_canadiens_2019, 'TOR': team_leafs_2019, 'OTT': team_senators_2019, 'TBL': team_lightning_2019, 'FLA': team_panthers_2019, 'BUF': team_sabres_2019, 'PHI': team_flyers_2019, 'CBJ': team_jackets_2019, 'NYR': team_rangers_2019, 'CAR': team_hurricanes_2019, 'NJD': team_devils_2019, 'NYI': team_islanders_2019, 'PIT': team_penguins_2019, 'WSH': team_capitals_2019}

In [109]:
warnings.filterwarnings("ignore")

strengths_2019=construct_team_skater_strengths(dict_teams_2019, normalized_df.loc[normalized_df['SEASON']<=2018])
team_goalie_strengths_2019=construct_team_goalie_strengths(dict_teams_goalies_2019, smaller_goalie_df.loc[smaller_goalie_df['SEASON']<=2018])

dict_overall_team_strength_2019={}
for i in strengths_2019:
    dict_overall_team_strength_2019[i]=strengths_2019[i]+team_goalie_strengths_2019[i]

result_dict_2019 = {new_key: dict_overall_team_strength_2019[old_key] for old_key, new_key in dict_const_mapping.items()}
sorted_dict_team_strengths_2019 = dict(sorted(result_dict_2019.items(), key=lambda item: item[1], reverse=True))
elo_2019=convert_to_elo_ratings(sorted_dict_team_strengths_2019)
elo_2019

1.6331559192370069
7.269788170051609
3.093331613882939
3.8129926560488574
6.078664174537926
5.286810977549894
3.8829065103401175
3.0397522205173226
1.9913044509646556
1.8095420919092666
3.2402331753366376
1.440211535955067
3.8670084031982728
2.9840260938629988
4.787572127134349
3.947159576661692
3.15311864809616
5.252015515550689
5.496838059014652
3.828507710655188
3.533624015787402
5.81948486686918
2.8584386913896593
1.5782087583423812
5.0283074054804535
0.7139899504981884
4.3597025874607445
1.902260089929133
3.025015877586238
0.8536501548206986
5.4877943312651905
2.1244059573325824
5.793024568411078
0.69968237611364
2.463720051771633
0.27631334783721523
5.760786473517426
1.9307153634964151
1.3774769243333684
3.0371293355564886
4.652132672597577
2.265253611582815
3.948666759184474
4.444230048702555
0.8547798435336332
1.7996044178765502
0.236058792076943
3.15681589095868
2.894412757735584
4.173077508335842
4.238695150641608
2.7503808542795847
5.81545957104458
2.9414069921942816
2.35898

{'Winnipeg Jets': 1596.8159736513776,
 'Boston Bruins': 1566.466937407459,
 'Tampa Bay Lightning': 1558.7132571652728,
 'Colorado Avalanche': 1551.6362908450192,
 'Vegas Golden Knights': 1543.568219891672,
 'Toronto Maple Leafs': 1538.7884099420176,
 'Philadelphia Flyers': 1522.3394512148027,
 'St. Louis Blues': 1521.381637755044,
 'Washington Capitals': 1520.1103660792232,
 'Nashville Predators': 1517.9461073177854,
 'San Jose Sharks': 1515.8000547039953,
 'Pittsburgh Penguins': 1511.5480233351584,
 'Columbus Blue Jackets': 1510.9899133748195,
 'New Jersey Devils': 1505.5771366540084,
 'Dallas Stars': 1500.9390131175041,
 'Calgary Flames': 1500.6176197322518,
 'New York Islanders': 1491.4609288656202,
 'Anaheim Ducks': 1490.4745260371762,
 'Florida Panthers': 1489.944513022185,
 'Arizona Coyotes': 1484.5165446538788,
 'Carolina Hurricanes': 1482.513887755089,
 'Montreal Canadiens': 1481.6189537751295,
 'Detroit Red Wings': 1479.3291798607886,
 'Los Angeles Kings': 1475.303776194321,
 

In [73]:
scores_df = pd.read_csv('/Users/hinayatali/Desktop/MIE368_PROJECT/team_proj/data/season_data.csv')
playoff_scores_df = pd.read_csv('/Users/hinayatali/Desktop/MIE368_PROJECT/team_proj/data/playoff_data.csv')
real_elo = pd.read_csv('/Users/hinayatali/Desktop/MIE368_PROJECT/team_proj/data/nhl_elo.csv')

In [99]:
K = 6

def playoff_tracker(simulation_pts, playoffs_made):

    conference_mapping = {
        'East': ['Florida Panthers', 'Toronto Maple Leafs', 'Tampa Bay Lightning', 'Boston Bruins', 'Buffalo Sabres', 'Detroit Red Wings',
                 'Ottawa Senators', 'Montreal Canadiens', 'Carolina Hurricanes', 'New York Rangers', 'Pittsburgh Penguins', 
                 'Washington Capitals', 'New York Islanders', 'Columbus Blue Jackets', 'New Jersey Devils', 'Philadelphia Flyers'],
        'West': ['Colorado Avalanche', 'Minnesota Wild', 'St. Louis Blues', 'Dallas Stars', 'Nashville Predators', 'Winnipeg Jets',
                 'Chicago Blackhawks', 'Arizona Coyotes', 'Calgary Flames', 'Edmonton Oilers', 'Los Angeles Kings', 'Vegas Golden Knights',
                 'Vancouver Canucks', 'San Jose Sharks', 'Anaheim Ducks', 'Seattle Kraken']
    }
    
    # Separate teams into East and West lists
    for conference, teams in conference_mapping.items():
        valid_teams = [i for i in teams if i in simulation_pts]
        sorted_teams = sorted(valid_teams, key=lambda x: simulation_pts[x], reverse=True)

        for team in sorted_teams[:8]:
            playoffs_made[team] += 1

    return playoffs_made

def season_forecaster(df, starting_elo):
        
    # Create a dictionary to track point totals
    simulation_pts = {team: 0 for team in df['Home'].unique()}

    for idx, row in df.iterrows():
        df, starting_elo, simulation_pts = forecast_game_elo(row['Home'], row['Visitor'], row['Season'], df, idx, starting_elo, simulation_pts)  
        
    return df, starting_elo, simulation_pts

def forecast_game_elo(home_team, away_team, season, df, idx, elo_ratings, simulation_wins):

    if home_team not in elo_ratings:
        elo_ratings[home_team] = 1500
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1500

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)

    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob
    
    home_win = np.random.choice([0, 1], p=[away_prob, home_prob])

    # Calculate base scores for the home and away team
    score_home = 2.845905 + 0.003584*Elo_diff_home
    score_away = 2.845905 + 0.003584*(-1)*Elo_diff_home

    conditions_satisfied = 0 

    while conditions_satisfied == 0:
        
        # poisson distribution to shift scores
        poisson_score_home = np.random.poisson(max(0, score_home))
        poisson_score_away = np.random.poisson(max(0, score_away))

        home_goal_diff = poisson_score_home - poisson_score_away
            
        # does it satsify conditions?
        if home_win == 1:
            if home_goal_diff > 0:
                conditions_satisfied = 1
        
        if home_win == 0:
            if home_goal_diff < 0: 
                conditions_satisfied = 1

    #------------------------------------------------------------------------#
    OT_choice = 0
    
    if abs(home_goal_diff) == 1:
        # Decide if the game goes to OT
        OT_choice = np.random.choice(a= [1, 0], p= [0.505, 0.495])
        
            
    if home_win == 1:
        winner_elo_diff = Elo_diff_home
        simulation_wins[home_team] += 2

        if OT_choice == 1:
            simulation_wins[away_team] += 1
        
    if home_win == 0:
        winner_elo_diff = -1*Elo_diff_home
        simulation_wins[away_team] += 2

        if OT_choice == 1:
            simulation_wins[home_team] += 1

    pre_g_fav_h = home_win - home_prob  
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)
    
    # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(poisson_score_home, poisson_score_away)

    # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h 
  
    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    df.at[idx, 'G_Home'] = poisson_score_home
    df.at[idx, 'G_Vis'] = poisson_score_away
    df.at[idx, 'OT Ind'] = OT_choice
    #--------------------------------------------------------------------------#

    return df, elo_ratings, simulation_wins

def margin_of_victory(home_goals, away_goals):
    
    mov = abs(home_goals - away_goals)
    mov_mult = 0.6686*np.log(mov)+0.8048
    
    return mov_mult

def calculate_prob_winning(home_team, away_team, elo_ratings):
    
    Elo_diff_home = elo_ratings[home_team]-elo_ratings[away_team] + 26  #add extra for home-ice advantage   
    #Elo_diff_away = elo_ratings[away_team] - elo_ratings[home_team]
    prob_win_home = 1/(10**(-1*Elo_diff_home/400)+1)
    #prob_win_away = 1/(10**(-1*Elo_diff_away/400)+1)
    prob_win_away = 1 - prob_win_home
    
    return prob_win_home, prob_win_away, Elo_diff_home

In [116]:
import copy


def get_results(season, elo_ratings_input, num_simulations):
    df2 = copy.deepcopy(scores_df[scores_df['Season'] == season])
    season_df = df2.copy()

    playoffs_counter = {team: 0 for team in df2['Home'].unique()}
    # Create a dictionary to track overall standings
    standings = {team: np.zeros(num_simulations) for team in df2['Home'].unique()}
    for simulation in range(num_simulations):
        # elo = copy.deepcopy(final_season_elos[2021])  # Creating a deep copy of final_season_elos[2021]
        elo_ratings = elo_ratings_input.copy()  # Initial copy for modifications

        season_2019, elo_ratings, simulation_pts = season_forecaster(season_df, elo_ratings)
        playoffs_counter = playoff_tracker(simulation_pts, playoffs_counter)
        for team, pts in simulation_pts.items():
            standings[team][simulation] = pts


    for team, pts in standings.items():
        mean_pts = np.mean(pts)
        std_deviation = np.std(pts)
        print(f"{team}: Mean point total = {mean_pts}, Standard Deviation = {std_deviation}")
    
    return standings

In [137]:
standings_results_2023=get_results(2023, elo_2022, 500)
mean_pts_2023 = pd.DataFrame(columns = ['PTS']).rename_axis('Team')
for team, pts in standings_results_2023.items():
    mean_pts_2023.loc[team] = [np.mean(pts)]

standings_results_2022=get_results(2022, elo_2021, 500)
mean_pts_2022 = pd.DataFrame(columns = ['PTS']).rename_axis('Team')
for team, pts in standings_results_2022.items():
    mean_pts_2022.loc[team] = [np.mean(pts)]

standings_results_2019=get_results(2019, elo_2019, 500)
mean_pts_2019 = pd.DataFrame(columns = ['PTS']).rename_axis('Team')
for team, pts in standings_results_2019.items():
    mean_pts_2019.loc[team] = [np.mean(pts)]

Nashville Predators: Mean point total = 96.092, Standard Deviation = 11.989309237816832
San Jose Sharks: Mean point total = 85.134, Standard Deviation = 12.318118525164465
Los Angeles Kings: Mean point total = 104.186, Standard Deviation = 10.964460953462327
New York Rangers: Mean point total = 100.068, Standard Deviation = 11.705356722458312
Anaheim Ducks: Mean point total = 75.532, Standard Deviation = 12.09202117100363
Carolina Hurricanes: Mean point total = 87.262, Standard Deviation = 12.228710316300734
Colorado Avalanche: Mean point total = 99.328, Standard Deviation = 11.211441298958846
Edmonton Oilers: Mean point total = 92.178, Standard Deviation = 12.166935357763679
Montreal Canadiens: Mean point total = 60.566, Standard Deviation = 11.263287441950506
Washington Capitals: Mean point total = 84.492, Standard Deviation = 12.59182020202004
Buffalo Sabres: Mean point total = 82.598, Standard Deviation = 12.601126775015002
Calgary Flames: Mean point total = 101.782, Standard Devia

In [139]:
# 2022-23 Season
real_scores_2023 = np.array([['Boston Bruins',135],['Toronto Maple Leafs',111],['Tampa Bay Lightning',98],
              ['Florida Panthers',92],['Buffalo Sabres',91],['Ottawa Senators',86],['Detroit Red Wings',80],
              ['Montreal Canadiens',68],['Carolina Hurricanes',113],['New Jersey Devils',112],['New York Rangers',107],
              ['New York Islanders',93],['Pittsburgh Penguins',91],['Washington Capitals',80],['Philadelphia Flyers',75],
              ['Columbus Blue Jackets',59],['Colorado Avalanche',109],['Dallas Stars',108],['Minnesota Wild',103],
              ['Winnipeg Jets',95],['Nashville Predators',92],['St. Louis Blues',81],['Arizona Coyotes',70],
              ['Chicago Blackhawks',59],['Vegas Golden Knights',111],['Edmonton Oilers',109],['Los Angeles Kings',104],
              ['Seattle Kraken',100],['Calgary Flames',93],['Vancouver Canucks',83],['San Jose Sharks',60],['Anaheim Ducks',58]])

playoffs_2023 = np.array([['Boston Bruins',1],['Toronto Maple Leafs',1],['Tampa Bay Lightning',1],
              ['Florida Panthers',1],['Buffalo Sabres',0],['Ottawa Senators',0],['Detroit Red Wings',0],
              ['Montreal Canadiens',0],['Carolina Hurricanes',1],['New Jersey Devils',1],['New York Rangers',1],
              ['New York Islanders',1],['Pittsburgh Penguins',0],['Washington Capitals',0],['Philadelphia Flyers',0],
              ['Columbus Blue Jackets',0],['Colorado Avalanche',1],['Dallas Stars',1],['Minnesota Wild',1],
              ['Winnipeg Jets',1],['Nashville Predators',0],['St. Louis Blues',0],['Arizona Coyotes',0],
              ['Chicago Blackhawks',0],['Vegas Golden Knights',1],['Edmonton Oilers',1],['Los Angeles Kings',1],
              ['Seattle Kraken',1],['Calgary Flames',0],['Vancouver Canucks',0],['San Jose Sharks',0],['Anaheim Ducks',0]])

real_scores_2023 = pd.DataFrame(real_scores_2023)
real_scores_2023 = real_scores_2023.rename(columns={0:'Team', 1:'PTS'})
real_scores_2023['PTS'] = pd.to_numeric(real_scores_2023['PTS'])

# 2021-22 Season
real_scores_2022 = np.array([['Florida Panthers',122],['Toronto Maple Leafs',115],['Tampa Bay Lightning',110],
              ['Boston Bruins',107],['Buffalo Sabres',75],['Detroit Red Wings',74],['Ottawa Senators',73],
              ['Montreal Canadiens',55],['Carolina Hurricanes',116],['New York Rangers',110],['Pittsburgh Penguins',103],
              ['Washington Capitals',100],['New York Islanders',84],['Columbus Blue Jackets',81],['New Jersey Devils',63],
              ['Philadelphia Flyers',61],['Colorado Avalanche',119],['Minnesota Wild',113],['St. Louis Blues',109],
              ['Dallas Stars',98],['Nashville Predators',97],['Winnipeg Jets',89],['Chicago Blackhawks',68],
              ['Arizona Coyotes',57],['Calgary Flames',111],['Edmonton Oilers',104],['Los Angeles Kings',99],
              ['Vegas Golden Knights',94],['Vancouver Canucks',92],['San Jose Sharks',77],['Anaheim Ducks',76],['Seattle Kraken',60]])

real_scores_2022 = pd.DataFrame(real_scores_2022)
real_scores_2022 = real_scores_2022.rename(columns={0:'Team', 1:'PTS'})
real_scores_2022['PTS'] = pd.to_numeric(real_scores_2022['PTS'])

#2018-19 Season
real_scores_2019 = np.array([['Tampa Bay Lightning',128],['Boston Bruins',107],['Toronto Maple Leafs',100],
              ['Montreal Canadiens',96],['Florida Panthers',86],['Buffalo Sabres',76],['Detroit Red Wings',74],
              ['Ottawa Senators',64],['Washington Capitals',104],['New York Islanders',103],['Pittsburgh Penguins',100],
              ['Carolina Hurricanes',99],['Columbus Blue Jackets',98],['Philadelphia Flyers',82],['New York Rangers',78],
              ['New Jersey Devils',72],['Nashville Predators',100],['Winnipeg Jets',99],['St. Louis Blues',99],
              ['Dallas Stars',93],['Colorado Avalanche',90],['Chicago Blackhawks',84],['Minnesota Wild',83],
              ['Calgary Flames',107],['San Jose Sharks',101],['Vegas Golden Knights',93],
              ['Vancouver Canucks',81],['Anaheim Ducks',80],['Edmonton Oilers',79],['Los Angeles Kings',71]])

real_scores_2019 = pd.DataFrame(real_scores_2019)
real_scores_2019 = real_scores_2019.rename(columns={0:'Team', 1:'PTS'})
real_scores_2019['PTS'] = pd.to_numeric(real_scores_2019['PTS'])

In [146]:
from sklearn.metrics import r2_score


testing_2023=real_scores_2023.merge(mean_pts_2023, how='left', on=['Team'])
print(abs(testing_2023['PTS_x']-testing_2023['PTS_y']).mean())

testing_2022=real_scores_2022.merge(mean_pts_2022, how='left', on=['Team'])
print(abs(testing_2022['PTS_x']-testing_2022['PTS_y']).mean())

testing_2019=real_scores_2019.merge(mean_pts_2019, how='left', on=['Team'])
print(abs(testing_2019['PTS_x']-testing_2019['PTS_y']).mean())

total_table=pd.concat([testing_2023, testing_2022, testing_2019], axis=0)

print(abs(total_table['PTS_x']-total_table['PTS_y']).mean())
print(r2_score(total_table['PTS_x'], total_table['PTS_y']))

12.669687499999998
13.744937499999999
8.1644
11.59787234042553
0.3102591451342158


In [143]:
testing_2019

,Team,PTS_x,PTS_y
0,Tampa Bay Lightning,128,102.800
1,Boston Bruins,107,104.356
2,Toronto Maple Leafs,100,98.204
3,Montreal Canadiens,96,86.060
4,Florida Panthers,86,88.338
5,Buffalo Sabres,76,83.194
6,Detroit Red Wings,74,85.040
7,Ottawa Senators,64,66.364
8,Washington Capitals,104,94.658
9,New York Islanders,103,88.700
